In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from scipy.stats import gaussian_kde
from sklearn.metrics import r2_score

from utils import *
from descriptor import *
import warnings

base_path = os.getcwd()
warnings.filterwarnings('ignore')
filename = "../data/features/soap_topology_total.csv"
data = pd.read_csv(filename,index_col=0)
data

,cif_file,topology,first_building_block,second_building_block_type,second_building_block,edge,f-chi-0-all,f-chi-1-all,f-chi-2-all,f-chi-3-all,f-Z-0-all,f-Z-1-all,f-Z-2-all,f-Z-3-all,f-I-0-all,f-I-1-all,f-I-2-all,f-I-3-all,f-T-0-all,f-T-1-all,f-T-2-all,f-T-3-all,f-S-0-all,f-S-1-all,f-S-2-all,f-S-3-all,mc-chi-0-all,mc-chi-1-all,mc-chi-2-all,mc-chi-3-all,mc-Z-0-all,mc-Z-1-all,mc-Z-2-all,mc-Z-3-all,mc-I-0-all,mc-I-1-all,mc-I-2-all,mc-I-3-all,mc-T-0-all,mc-T-1-all,mc-T-2-all,mc-T-3-all,mc-S-0-all,mc-S-1-all,mc-S-2-all,mc-S-3-all,D_mc-chi-0-all,D_mc-chi-1-all,D_mc-chi-2-all,D_mc-chi-3-all,D_mc-Z-0-all,D_mc-Z-1-all,D_mc-Z-2-all,D_mc-Z-3-all,D_mc-I-0-all,D_mc-I-1-all,D_mc-I-2-all,D_mc-I-3-all,D_mc-T-0-all,D_mc-T-1-all,D_mc-T-2-all,D_mc-T-3-all,D_mc-S-0-all,D_mc-S-1-all,D_mc-S-2-all,D_mc-S-3-all,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-I-0,f-lig-I-1,f-lig-I-2,f-lig-I-3,f-lig-T-0,f-lig-T-1,f-lig-T-2,f-lig-T-3,f-lig-S-0,f-lig-S-1,f-lig-S-2,f-lig-S-3,lc-chi-0-all,lc-chi-1-all,lc-chi-2-all,lc-chi-3-all,lc-Z-0-all,lc-Z-1-all,lc-Z-2-all,lc-Z-3-all,lc-I-0-all,lc-I-1-all,lc-I-2-all,lc-I-3-all,lc-T-0-all,lc-T-1-all,lc-T-2-all,lc-T-3-all,lc-S-0-all,lc-S-1-all,lc-S-2-all,lc-S-3-all,lc-alpha-0-all,lc-alpha-1-all,lc-alpha-2-all,lc-alpha-3-all,D_lc-chi-0-all,D_lc-chi-1-all,D_lc-chi-2-all,D_lc-chi-3-all,D_lc-Z-0-all,D_lc-Z-1-all,D_lc-Z-2-all,D_lc-Z-3-all,D_lc-I-0-all,D_lc-I-1-all,D_lc-I-2-all,D_lc-I-3-all,D_lc-T-0-all,D_lc-T-1-all,D_lc-T-2-all,D_lc-T-3-all,D_lc-S-0-all,D_lc-S-1-all,D_lc-S-2-all,D_lc-S-3-all,D_lc-alpha-0-all,D_lc-alpha-1-all,D_lc-alpha-2-all,D_lc-alpha-3-all,func-chi-0-all,func-chi-1-all,func-chi-2-all,func-chi-3-all,func-Z-0-all,func-Z-1-all,func-Z-2-all,func-Z-3-all,func-I-0-all,func-I-1-all,func-I-2-all,func-I-3-all,func-T-0-all,func-T-1-all,func-T-2-all,func-T-3-all,func-S-0-all,func-S-1-all,func-S-2-all,func-S-3-all,func-alpha-0-all,func-alpha-1-all,func-alpha-2-all,func-alpha-3-all,D_func-chi-0-all,D_func-chi-1-all,D_func-chi-2-all,D_func-chi-3-all,D_func-Z-0-all,D_func-Z-1-all,D_func-Z-2-all,D_func-Z-3-all,D_func-I-0-all,D_func-I-1-all,D_func-I-2-all,D_func-I-3-all,D_func-T-0-all,D_func-T-1-all,D_func-T-2-all,D_func-T-3-all,D_func-S-0-all,D_func-S-1-all,D_func-S-2-all,D_func-S-3-all,D_func-alpha-0-all,D_func-alpha-1-all,D_func-alpha-2-all,D_func-alpha-3-all,Di,Df,Dif,rho,VSA,GSA,VPOV,GPOV,POAV_vol_frac,PONAV_vol_frac,GPOAV,GPONAV,POAV,PONAV,KVRH,GVRH,soap0,soap1,soap2,soap3,soap4,soap5,soap6,soap7,soap8,soap9,soap10,soap11,soap12,soap13,soap14,soap15,soap16,soap17,soap18,soap19,soap20,soap21,soap22,soap23,soap24,soap25,soap26,soap27,soap28,soap29,soap30,soap31,soap32,soap33,soap34,soap35,soap36,soap37,soap38,soap39,soap40,soap41,soap42,soap43,soap44,soap45,soap46,soap47,soap48,soap49,soap50,soap51,...,soap7310,soap7311,soap7312,soap7313,soap7314,soap7315,soap7316,soap7317,soap7318,soap7319,soap7320,soap7321,soap7322,soap7323,soap7324,soap7325,soap7326,soap7327,soap7328,soap7329,soap7330,soap7331,soap7332,soap7333,soap7334,soap7335,soap7336,soap7337,soap7338,soap7339,soap7340,soap7341,soap7342,soap7343,soap7344,soap7345,soap7346,soap7347,soap7348,soap7349,soap7350,soap7351,soap7352,soap7353,soap7354,soap7355,soap7356,soap7357,soap7358,soap7359,soap7360,soap7361,soap7362,soap7363,soap7364,soap7365,soap7366,soap7367,soap7368,soap7369,soap7370,soap7371,soap7372,soap7373,soap7374,soap7375,soap7376,soap7377,soap7378,soap7379,soap7380,soap7381,soap7382,soap7383,soap7384,soap7385,soap7386,soap7387,soap7388,soap7389,soap7390,soap7391,soap7392,soap7393,soap7394,soap7395,soap7396,soap7397,soap7398,soap7399,soap7400,soap7401,soap7402,soap7403,soap7404,soap7405,soap7406,soap7407,soap7408,soap7409,soap7410,soap7411,soap7412,soap7413,soap7414,soap7415,soap7416,soap7417,soap7418,soap7419,soap7420,soap7421,soap7422,soap7423,soap7424,soap7425,soap7426,soap7427,soap7428,soap7429,soap7430,soap7431,soap7432,soap7433,soap7434,soap7435,soap7436,soap7437,soap7438,soap7439,soap7440,soap7441,soap7442,

In [2]:
from sklearn.preprocessing import OneHotEncoder
topology = np.array(data["topology"])
topology = topology.reshape(-1,1)
topology_encoder = OneHotEncoder()
topology_en_data = topology_encoder.fit_transform(topology).toarray()

In [3]:
property_ = ["GVRH"]
columns = geometric + property_ 
descriptors = geometric 
chosen = data[columns]
category = pd.DataFrame(topology_en_data)
chosen = pd.concat([chosen,category],axis=1)
chosen

,Di,Df,Dif,rho,VSA,GSA,VPOV,GPOV,POAV_vol_frac,PONAV_vol_frac,GPOAV,GPONAV,POAV,PONAV,GVRH,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,...,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501
0,9.64336,8.80746,9.64336,3146.50,2193.800,3128.66,0.6635,0.946242,0.6635,0.0,0.946242,0.0,2087.70,0.0,11.231780,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9.94957,9.48351,

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor


In [5]:
topology_index = list(chosen.columns[15:])

In [6]:
X = chosen[topology_index + geometric]
y = chosen[property_]

In [7]:
train_size = 1000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)



Learning rate set to 0.040943
0:	learn: 3.4106813	total: 49.9ms	remaining: 49.8s
1:	learn: 3.3656461	total: 53ms	remaining: 26.4s
2:	learn: 3.3221727	total: 55.8ms	remaining: 18.5s
3:	learn: 3.2782881	total: 58.5ms	remaining: 14.6s
4:	learn: 3.2385391	total: 61.2ms	remaining: 12.2s
5:	learn: 3.1984301	total: 64.1ms	remaining: 10.6s
6:	learn: 3.1609171	total: 66ms	remaining: 9.37s
7:	learn: 3.1272532	total: 68.2ms	remaining: 8.45s
8:	learn: 3.0941796	total: 70.3ms	remaining: 7.74s
9:	learn: 3.0664312	total: 72.5ms	remaining: 7.17s
10:	learn: 3.0369610	total: 74.4ms	remaining: 6.69s
11:	learn: 3.0054096	total: 76.1ms	remaining: 6.27s
12:	learn: 2.9806296	total: 77.9ms	remaining: 5.92s
13:	learn: 2.9578491	total: 78.9ms	remaining: 5.56s
14:	learn: 2.9348265	total: 80.8ms	remaining: 5.3s
15:	learn: 2.9104006	total: 82.5ms	remaining: 5.08s
16:	learn: 2.8881312	total: 84.2ms	remaining: 4.87s
17:	learn: 2.8638571	total: 85.7ms	remaining: 4.67s
18:	learn: 2.8459913	total: 87.1ms	remaining: 4.5

344:	learn: 1.6819605	total: 383ms	remaining: 727ms
345:	learn: 1.6807432	total: 384ms	remaining: 726ms
346:	learn: 1.6800411	total: 385ms	remaining: 725ms
347:	learn: 1.6763157	total: 386ms	remaining: 723ms
348:	learn: 1.6742342	total: 387ms	remaining: 721ms
349:	learn: 1.6733705	total: 388ms	remaining: 720ms
350:	learn: 1.6725204	total: 389ms	remaining: 718ms
351:	learn: 1.6711845	total: 389ms	remaining: 717ms
352:	learn: 1.6704855	total: 390ms	remaining: 715ms
353:	learn: 1.6697647	total: 391ms	remaining: 714ms
354:	learn: 1.6672683	total: 392ms	remaining: 712ms
355:	learn: 1.6664355	total: 393ms	remaining: 711ms
356:	learn: 1.6652711	total: 394ms	remaining: 709ms
357:	learn: 1.6645982	total: 395ms	remaining: 708ms
358:	learn: 1.6635554	total: 396ms	remaining: 706ms
359:	learn: 1.6616536	total: 396ms	remaining: 705ms
360:	learn: 1.6598794	total: 397ms	remaining: 703ms
361:	learn: 1.6591834	total: 398ms	remaining: 702ms
362:	learn: 1.6550337	total: 399ms	remaining: 700ms
363:	learn: 

564:	learn: 1.3878490	total: 574ms	remaining: 442ms
565:	learn: 1.3868064	total: 575ms	remaining: 441ms
566:	learn: 1.3853324	total: 576ms	remaining: 440ms
567:	learn: 1.3849446	total: 577ms	remaining: 439ms
568:	learn: 1.3843859	total: 578ms	remaining: 438ms
569:	learn: 1.3835001	total: 579ms	remaining: 437ms
570:	learn: 1.3821510	total: 580ms	remaining: 435ms
571:	learn: 1.3805619	total: 580ms	remaining: 434ms
572:	learn: 1.3796014	total: 581ms	remaining: 433ms
573:	learn: 1.3791813	total: 582ms	remaining: 432ms
574:	learn: 1.3779126	total: 583ms	remaining: 431ms
575:	learn: 1.3769285	total: 584ms	remaining: 430ms
576:	learn: 1.3756956	total: 585ms	remaining: 429ms
577:	learn: 1.3732290	total: 586ms	remaining: 428ms
578:	learn: 1.3728512	total: 586ms	remaining: 426ms
579:	learn: 1.3711060	total: 587ms	remaining: 425ms
580:	learn: 1.3705516	total: 588ms	remaining: 424ms
581:	learn: 1.3702078	total: 589ms	remaining: 423ms
582:	learn: 1.3697365	total: 590ms	remaining: 422ms
583:	learn: 

786:	learn: 1.1622423	total: 765ms	remaining: 207ms
787:	learn: 1.1619981	total: 766ms	remaining: 206ms
788:	learn: 1.1617602	total: 767ms	remaining: 205ms
789:	learn: 1.1613060	total: 768ms	remaining: 204ms
790:	learn: 1.1600429	total: 769ms	remaining: 203ms
791:	learn: 1.1598323	total: 769ms	remaining: 202ms
792:	learn: 1.1595849	total: 770ms	remaining: 201ms
793:	learn: 1.1583982	total: 771ms	remaining: 200ms
794:	learn: 1.1577596	total: 772ms	remaining: 199ms
795:	learn: 1.1572888	total: 773ms	remaining: 198ms
796:	learn: 1.1556802	total: 774ms	remaining: 197ms
797:	learn: 1.1554457	total: 775ms	remaining: 196ms
798:	learn: 1.1538805	total: 775ms	remaining: 195ms
799:	learn: 1.1533396	total: 776ms	remaining: 194ms
800:	learn: 1.1526929	total: 777ms	remaining: 193ms
801:	learn: 1.1495946	total: 778ms	remaining: 192ms
802:	learn: 1.1480607	total: 779ms	remaining: 191ms
803:	learn: 1.1476977	total: 780ms	remaining: 190ms
804:	learn: 1.1474400	total: 781ms	remaining: 189ms
805:	learn: 

Learning rate set to 0.040943
0:	learn: 3.2964280	total: 2.3ms	remaining: 2.3s
1:	learn: 3.2521435	total: 4.14ms	remaining: 2.07s
2:	learn: 3.2050638	total: 5.97ms	remaining: 1.98s
3:	learn: 3.1584718	total: 7.92ms	remaining: 1.97s
4:	learn: 3.1230154	total: 9.82ms	remaining: 1.95s
5:	learn: 3.0844836	total: 11.6ms	remaining: 1.92s
6:	learn: 3.0503557	total: 13.3ms	remaining: 1.89s
7:	learn: 3.0150327	total: 15.1ms	remaining: 1.88s
8:	learn: 2.9818981	total: 16.7ms	remaining: 1.84s
9:	learn: 2.9481558	total: 18.4ms	remaining: 1.82s
10:	learn: 2.9176186	total: 20.2ms	remaining: 1.81s
11:	learn: 2.8888032	total: 21.7ms	remaining: 1.79s
12:	learn: 2.8604836	total: 23.3ms	remaining: 1.76s
13:	learn: 2.8346937	total: 24.8ms	remaining: 1.75s
14:	learn: 2.8045632	total: 26.3ms	remaining: 1.73s
15:	learn: 2.7794977	total: 27.7ms	remaining: 1.7s
16:	learn: 2.7559233	total: 29.2ms	remaining: 1.69s
17:	learn: 2.7325155	total: 30.7ms	remaining: 1.67s
18:	learn: 2.7095661	total: 31.9ms	remaining: 1

...

Learning rate set to 0.040943
0:	learn: 3.3373932	total: 3.1ms	remaining: 3.09s
1:	learn: 3.2956193	total: 5.89ms	remaining: 2.94s
2:	learn: 3.2563045	total: 8.53ms	remaining: 2.83s
3:	learn: 3.2164821	total: 10.8ms	remaining: 2.69s
4:	learn: 3.1764271	total: 13.5ms	remaining: 2.69s
5:	learn: 3.1347194	total: 16.4ms	remaining: 2.72s
6:	learn: 3.1029286	total: 17.9ms	remaining: 2.54s
7:	learn: 3.0702208	total: 19.8ms	remaining: 2.46s
8:	learn: 3.0392694	total: 21.8ms	remaining: 2.4s
9:	learn: 3.0129696	total: 23.6ms	remaining: 2.33s
10:	learn: 2.9791742	total: 25.5ms	remaining: 2.29s
11:	learn: 2.9688807	total: 27.2ms	remaining: 2.24s
12:	learn: 2.9409228	total: 28.8ms	remaining: 2.18s
13:	learn: 2.9102407	total: 30.3ms	remaining: 2.14s
14:	learn: 2.8793550	total: 31.9ms	remaining: 2.1s
15:	learn: 2.8670223	total: 33ms	remaining: 2.03s
16:	learn: 2.8428698	total: 34.5ms	remaining: 2s
17:	learn: 2.8188063	total: 36ms	remaining: 1.97s
18:	learn: 2.7925787	total: 37.5ms	remaining: 1.94s
19

182:	learn: 1.8360191	total: 190ms	remaining: 849ms
183:	learn: 1.8333020	total: 191ms	remaining: 848ms
184:	learn: 1.8303695	total: 192ms	remaining: 846ms
185:	learn: 1.8281579	total: 193ms	remaining: 844ms
186:	learn: 1.8256636	total: 194ms	remaining: 843ms
187:	learn: 1.8218987	total: 195ms	remaining: 841ms
188:	learn: 1.8189236	total: 196ms	remaining: 839ms
189:	learn: 1.8150882	total: 196ms	remaining: 837ms
190:	learn: 1.8126852	total: 197ms	remaining: 835ms
191:	learn: 1.8094078	total: 198ms	remaining: 833ms
192:	learn: 1.8067938	total: 199ms	remaining: 831ms
193:	learn: 1.8041300	total: 200ms	remaining: 829ms
194:	learn: 1.8018963	total: 200ms	remaining: 828ms
195:	learn: 1.7990093	total: 201ms	remaining: 826ms
196:	learn: 1.7953278	total: 202ms	remaining: 824ms
197:	learn: 1.7926217	total: 203ms	remaining: 823ms
198:	learn: 1.7906268	total: 204ms	remaining: 821ms
199:	learn: 1.7850192	total: 205ms	remaining: 819ms
200:	learn: 1.7823250	total: 206ms	remaining: 818ms
201:	learn: 

401:	learn: 1.4510658	total: 381ms	remaining: 567ms
402:	learn: 1.4502523	total: 382ms	remaining: 566ms
403:	learn: 1.4496192	total: 383ms	remaining: 565ms
404:	learn: 1.4441943	total: 384ms	remaining: 563ms
405:	learn: 1.4437283	total: 384ms	remaining: 562ms
406:	learn: 1.4430680	total: 385ms	remaining: 561ms
407:	learn: 1.4425295	total: 386ms	remaining: 560ms
408:	learn: 1.4407509	total: 387ms	remaining: 559ms
409:	learn: 1.4387398	total: 388ms	remaining: 558ms
410:	learn: 1.4380419	total: 389ms	remaining: 557ms
411:	learn: 1.4370084	total: 390ms	remaining: 556ms
412:	learn: 1.4342183	total: 391ms	remaining: 555ms
413:	learn: 1.4335630	total: 391ms	remaining: 554ms
414:	learn: 1.4326852	total: 392ms	remaining: 553ms
415:	learn: 1.4314198	total: 393ms	remaining: 552ms
416:	learn: 1.4305759	total: 394ms	remaining: 551ms
417:	learn: 1.4300879	total: 395ms	remaining: 550ms
418:	learn: 1.4296460	total: 396ms	remaining: 549ms
419:	learn: 1.4284170	total: 397ms	remaining: 548ms
420:	learn: 

619:	learn: 1.1701063	total: 573ms	remaining: 351ms
620:	learn: 1.1697801	total: 574ms	remaining: 350ms
621:	learn: 1.1676879	total: 575ms	remaining: 349ms
622:	learn: 1.1659453	total: 576ms	remaining: 348ms
623:	learn: 1.1650467	total: 576ms	remaining: 347ms
624:	learn: 1.1628046	total: 577ms	remaining: 346ms
625:	learn: 1.1625108	total: 578ms	remaining: 345ms
626:	learn: 1.1617930	total: 579ms	remaining: 344ms
627:	learn: 1.1600895	total: 580ms	remaining: 344ms
628:	learn: 1.1597913	total: 581ms	remaining: 343ms
629:	learn: 1.1591597	total: 582ms	remaining: 342ms
630:	learn: 1.1584578	total: 582ms	remaining: 341ms
631:	learn: 1.1567306	total: 583ms	remaining: 340ms
632:	learn: 1.1549062	total: 584ms	remaining: 339ms
633:	learn: 1.1545890	total: 585ms	remaining: 338ms
634:	learn: 1.1522890	total: 586ms	remaining: 337ms
635:	learn: 1.1517493	total: 587ms	remaining: 336ms
636:	learn: 1.1501329	total: 588ms	remaining: 335ms
637:	learn: 1.1496886	total: 589ms	remaining: 334ms
638:	learn: 

834:	learn: 0.9784839	total: 763ms	remaining: 151ms
835:	learn: 0.9771896	total: 764ms	remaining: 150ms
836:	learn: 0.9755357	total: 765ms	remaining: 149ms
837:	learn: 0.9750577	total: 766ms	remaining: 148ms
838:	learn: 0.9729094	total: 767ms	remaining: 147ms
839:	learn: 0.9714392	total: 768ms	remaining: 146ms
840:	learn: 0.9710349	total: 769ms	remaining: 145ms
841:	learn: 0.9708309	total: 769ms	remaining: 144ms
842:	learn: 0.9706102	total: 770ms	remaining: 143ms
843:	learn: 0.9678915	total: 771ms	remaining: 143ms
844:	learn: 0.9669522	total: 772ms	remaining: 142ms
845:	learn: 0.9667529	total: 773ms	remaining: 141ms
846:	learn: 0.9656283	total: 774ms	remaining: 140ms
847:	learn: 0.9640470	total: 775ms	remaining: 139ms
848:	learn: 0.9629841	total: 776ms	remaining: 138ms
849:	learn: 0.9620570	total: 776ms	remaining: 137ms
850:	learn: 0.9617415	total: 777ms	remaining: 136ms
851:	learn: 0.9615219	total: 778ms	remaining: 135ms
852:	learn: 0.9605076	total: 779ms	remaining: 134ms
853:	learn: 

In [8]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.1865062852700607
1.1987940484504045
1.192098097469565
1.2115366030271606
1.156569116126721
1.1735335612764528
0.40658913123877605
0.38622415907028984
0.41581177347206577
0.4285880097167931
0.3553221537801069
0.4469995601546246


In [9]:
train_size = 2000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.045682
0:	learn: 3.1416288	total: 3.97ms	remaining: 3.96s
1:	learn: 3.0965362	total: 7.7ms	remaining: 3.84s
2:	learn: 3.0569661	total: 11ms	remaining: 3.65s
3:	learn: 3.0193797	total: 13.9ms	remaining: 3.47s
4:	learn: 2.9857745	total: 16.7ms	remaining: 3.33s
5:	learn: 2.9509478	total: 19.7ms	remaining: 3.26s
6:	learn: 2.9194575	total: 22.2ms	remaining: 3.15s
7:	learn: 2.8875093	total: 24.5ms	remaining: 3.04s
8:	learn: 2.8629276	total: 26.9ms	remaining: 2.96s
9:	learn: 2.8367179	total: 29.6ms	remaining: 2.92s
10:	learn: 2.8141219	total: 31.7ms	remaining: 2.85s
11:	learn: 2.7905804	total: 33.7ms	remaining: 2.77s
12:	learn: 2.7726731	total: 36ms	remaining: 2.73s
13:	learn: 2.7514940	total: 38.2ms	remaining: 2.69s
14:	learn: 2.7334804	total: 40.2ms	remaining: 2.64s
15:	learn: 2.7153781	total: 42.2ms	remaining: 2.59s
16:	learn: 2.6973059	total: 44.2ms	remaining: 2.55s
17:	learn: 2.6830599	total: 46.2ms	remaining: 2.52s
18:	learn: 2.6666788	total: 48.3ms	remaining: 2.4

304:	learn: 1.8885071	total: 385ms	remaining: 878ms
305:	learn: 1.8876832	total: 386ms	remaining: 876ms
306:	learn: 1.8855857	total: 388ms	remaining: 875ms
307:	learn: 1.8842342	total: 389ms	remaining: 873ms
308:	learn: 1.8836420	total: 390ms	remaining: 871ms
309:	learn: 1.8825152	total: 391ms	remaining: 870ms
310:	learn: 1.8817130	total: 392ms	remaining: 869ms
311:	learn: 1.8810098	total: 393ms	remaining: 867ms
312:	learn: 1.8804270	total: 394ms	remaining: 865ms
313:	learn: 1.8790306	total: 395ms	remaining: 864ms
314:	learn: 1.8776577	total: 396ms	remaining: 862ms
315:	learn: 1.8765426	total: 398ms	remaining: 861ms
316:	learn: 1.8752000	total: 399ms	remaining: 859ms
317:	learn: 1.8718851	total: 400ms	remaining: 857ms
318:	learn: 1.8695445	total: 401ms	remaining: 856ms
319:	learn: 1.8688533	total: 402ms	remaining: 854ms
320:	learn: 1.8682772	total: 403ms	remaining: 853ms
321:	learn: 1.8670100	total: 404ms	remaining: 851ms
322:	learn: 1.8657058	total: 405ms	remaining: 850ms
323:	learn: 

478:	learn: 1.6519442	total: 578ms	remaining: 629ms
479:	learn: 1.6495194	total: 579ms	remaining: 628ms
480:	learn: 1.6485254	total: 580ms	remaining: 626ms
481:	learn: 1.6475884	total: 582ms	remaining: 625ms
482:	learn: 1.6472087	total: 583ms	remaining: 624ms
483:	learn: 1.6451484	total: 584ms	remaining: 622ms
484:	learn: 1.6447754	total: 585ms	remaining: 621ms
485:	learn: 1.6441983	total: 586ms	remaining: 620ms
486:	learn: 1.6420175	total: 587ms	remaining: 619ms
487:	learn: 1.6398743	total: 588ms	remaining: 617ms
488:	learn: 1.6384053	total: 589ms	remaining: 616ms
489:	learn: 1.6380367	total: 590ms	remaining: 615ms
490:	learn: 1.6372032	total: 592ms	remaining: 613ms
491:	learn: 1.6349359	total: 593ms	remaining: 612ms
492:	learn: 1.6344057	total: 594ms	remaining: 611ms
493:	learn: 1.6329284	total: 595ms	remaining: 609ms
494:	learn: 1.6317819	total: 596ms	remaining: 608ms
495:	learn: 1.6314203	total: 597ms	remaining: 607ms
496:	learn: 1.6307101	total: 598ms	remaining: 606ms
497:	learn: 

652:	learn: 1.4752810	total: 773ms	remaining: 411ms
653:	learn: 1.4741303	total: 774ms	remaining: 409ms
654:	learn: 1.4721948	total: 775ms	remaining: 408ms
655:	learn: 1.4706198	total: 776ms	remaining: 407ms
656:	learn: 1.4689158	total: 777ms	remaining: 406ms
657:	learn: 1.4678480	total: 778ms	remaining: 404ms
658:	learn: 1.4675687	total: 779ms	remaining: 403ms
659:	learn: 1.4661086	total: 780ms	remaining: 402ms
660:	learn: 1.4656732	total: 781ms	remaining: 401ms
661:	learn: 1.4653995	total: 782ms	remaining: 399ms
662:	learn: 1.4642995	total: 783ms	remaining: 398ms
663:	learn: 1.4640427	total: 785ms	remaining: 397ms
664:	learn: 1.4621897	total: 786ms	remaining: 396ms
665:	learn: 1.4613707	total: 787ms	remaining: 395ms
666:	learn: 1.4606824	total: 788ms	remaining: 393ms
667:	learn: 1.4604280	total: 789ms	remaining: 392ms
668:	learn: 1.4586421	total: 790ms	remaining: 391ms
669:	learn: 1.4583773	total: 791ms	remaining: 390ms
670:	learn: 1.4581092	total: 792ms	remaining: 388ms
671:	learn: 

825:	learn: 1.3317357	total: 965ms	remaining: 203ms
826:	learn: 1.3308507	total: 966ms	remaining: 202ms
827:	learn: 1.3292316	total: 967ms	remaining: 201ms
828:	learn: 1.3290249	total: 968ms	remaining: 200ms
829:	learn: 1.3288277	total: 970ms	remaining: 199ms
830:	learn: 1.3285996	total: 971ms	remaining: 197ms
831:	learn: 1.3280957	total: 972ms	remaining: 196ms
832:	learn: 1.3270926	total: 973ms	remaining: 195ms
833:	learn: 1.3258354	total: 974ms	remaining: 194ms
834:	learn: 1.3253122	total: 975ms	remaining: 193ms
835:	learn: 1.3249806	total: 976ms	remaining: 192ms
836:	learn: 1.3247735	total: 977ms	remaining: 190ms
837:	learn: 1.3243921	total: 978ms	remaining: 189ms
838:	learn: 1.3235208	total: 980ms	remaining: 188ms
839:	learn: 1.3230094	total: 981ms	remaining: 187ms
840:	learn: 1.3228154	total: 982ms	remaining: 186ms
841:	learn: 1.3223873	total: 983ms	remaining: 184ms
842:	learn: 1.3215870	total: 984ms	remaining: 183ms
843:	learn: 1.3210928	total: 985ms	remaining: 182ms
844:	learn: 

...

299:	learn: 1.7120505	total: 386ms	remaining: 901ms
300:	learn: 1.7114605	total: 387ms	remaining: 900ms
301:	learn: 1.7086991	total: 388ms	remaining: 898ms
302:	learn: 1.7065715	total: 390ms	remaining: 896ms
303:	learn: 1.7027819	total: 391ms	remaining: 895ms
304:	learn: 1.6990320	total: 392ms	remaining: 893ms
305:	learn: 1.6968887	total: 393ms	remaining: 891ms
306:	learn: 1.6918324	total: 394ms	remaining: 890ms
307:	learn: 1.6904036	total: 395ms	remaining: 888ms
308:	learn: 1.6878453	total: 396ms	remaining: 887ms
309:	learn: 1.6867003	total: 398ms	remaining: 885ms
310:	learn: 1.6858886	total: 399ms	remaining: 883ms
311:	learn: 1.6843710	total: 400ms	remaining: 882ms
312:	learn: 1.6834826	total: 401ms	remaining: 880ms
313:	learn: 1.6829155	total: 402ms	remaining: 878ms
314:	learn: 1.6815959	total: 403ms	remaining: 877ms
315:	learn: 1.6771055	total: 404ms	remaining: 875ms
316:	learn: 1.6741536	total: 405ms	remaining: 873ms
317:	learn: 1.6719906	total: 406ms	remaining: 872ms
318:	learn: 

472:	learn: 1.4624874	total: 579ms	remaining: 645ms
473:	learn: 1.4606904	total: 580ms	remaining: 644ms
474:	learn: 1.4598744	total: 581ms	remaining: 642ms
475:	learn: 1.4595032	total: 582ms	remaining: 641ms
476:	learn: 1.4586134	total: 583ms	remaining: 640ms
477:	learn: 1.4579191	total: 584ms	remaining: 638ms
478:	learn: 1.4567541	total: 586ms	remaining: 637ms
479:	learn: 1.4562086	total: 587ms	remaining: 635ms
480:	learn: 1.4530324	total: 588ms	remaining: 634ms
481:	learn: 1.4520048	total: 589ms	remaining: 633ms
482:	learn: 1.4504609	total: 590ms	remaining: 631ms
483:	learn: 1.4498173	total: 591ms	remaining: 630ms
484:	learn: 1.4481332	total: 592ms	remaining: 629ms
485:	learn: 1.4477676	total: 593ms	remaining: 627ms
486:	learn: 1.4468266	total: 594ms	remaining: 626ms
487:	learn: 1.4450495	total: 595ms	remaining: 625ms
488:	learn: 1.4443185	total: 596ms	remaining: 623ms
489:	learn: 1.4433387	total: 597ms	remaining: 622ms
490:	learn: 1.4429807	total: 599ms	remaining: 620ms
491:	learn: 

645:	learn: 1.2889021	total: 772ms	remaining: 423ms
646:	learn: 1.2873937	total: 773ms	remaining: 422ms
647:	learn: 1.2868921	total: 774ms	remaining: 420ms
648:	learn: 1.2851916	total: 775ms	remaining: 419ms
649:	learn: 1.2848059	total: 776ms	remaining: 418ms
650:	learn: 1.2840629	total: 777ms	remaining: 417ms
651:	learn: 1.2837231	total: 778ms	remaining: 415ms
652:	learn: 1.2825253	total: 779ms	remaining: 414ms
653:	learn: 1.2822649	total: 781ms	remaining: 413ms
654:	learn: 1.2817325	total: 782ms	remaining: 412ms
655:	learn: 1.2810570	total: 783ms	remaining: 411ms
656:	learn: 1.2799534	total: 784ms	remaining: 409ms
657:	learn: 1.2792884	total: 785ms	remaining: 408ms
658:	learn: 1.2783711	total: 786ms	remaining: 407ms
659:	learn: 1.2777357	total: 787ms	remaining: 406ms
660:	learn: 1.2771145	total: 788ms	remaining: 404ms
661:	learn: 1.2764789	total: 789ms	remaining: 403ms
662:	learn: 1.2752947	total: 791ms	remaining: 402ms
663:	learn: 1.2750353	total: 792ms	remaining: 401ms
664:	learn: 

819:	learn: 1.1594264	total: 965ms	remaining: 212ms
820:	learn: 1.1592305	total: 966ms	remaining: 211ms
821:	learn: 1.1587433	total: 967ms	remaining: 209ms
822:	learn: 1.1582712	total: 968ms	remaining: 208ms
823:	learn: 1.1578483	total: 969ms	remaining: 207ms
824:	learn: 1.1566243	total: 970ms	remaining: 206ms
825:	learn: 1.1560170	total: 971ms	remaining: 205ms
826:	learn: 1.1556057	total: 973ms	remaining: 203ms
827:	learn: 1.1554105	total: 974ms	remaining: 202ms
828:	learn: 1.1544126	total: 975ms	remaining: 201ms
829:	learn: 1.1542196	total: 976ms	remaining: 200ms
830:	learn: 1.1537128	total: 977ms	remaining: 199ms
831:	learn: 1.1528646	total: 978ms	remaining: 198ms
832:	learn: 1.1512668	total: 979ms	remaining: 196ms
833:	learn: 1.1502073	total: 981ms	remaining: 195ms
834:	learn: 1.1483608	total: 982ms	remaining: 194ms
835:	learn: 1.1468932	total: 983ms	remaining: 193ms
836:	learn: 1.1465173	total: 984ms	remaining: 192ms
837:	learn: 1.1458586	total: 985ms	remaining: 190ms
838:	learn: 

992:	learn: 1.0512461	total: 1.16s	remaining: 8.15ms
993:	learn: 1.0504958	total: 1.16s	remaining: 6.99ms
994:	learn: 1.0501708	total: 1.16s	remaining: 5.82ms
995:	learn: 1.0490161	total: 1.16s	remaining: 4.66ms
996:	learn: 1.0487381	total: 1.16s	remaining: 3.49ms
997:	learn: 1.0485781	total: 1.16s	remaining: 2.33ms
998:	learn: 1.0471001	total: 1.16s	remaining: 1.16ms
999:	learn: 1.0455593	total: 1.16s	remaining: 0us


In [10]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.1527698850778294
1.1239207612927136
1.1510735380460213
1.1743292265076575
1.1558224926702094
1.1587034068725466
0.45439319197720945
0.48008237130185616
0.47024434837719864
0.4591295466565415
0.38169733901007574
0.4808123545403753


In [11]:
train_size = 3000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.048704
0:	learn: 3.1934054	total: 3.08ms	remaining: 3.08s
1:	learn: 3.1468224	total: 6.3ms	remaining: 3.15s
2:	learn: 3.1018293	total: 9.85ms	remaining: 3.27s
3:	learn: 3.0621695	total: 13.2ms	remaining: 3.28s
4:	learn: 3.0242835	total: 16ms	remaining: 3.18s
5:	learn: 2.9902516	total: 18.7ms	remaining: 3.1s
6:	learn: 2.9566093	total: 21.6ms	remaining: 3.06s
7:	learn: 2.9241366	total: 24.3ms	remaining: 3.01s
8:	learn: 2.8958878	total: 26.9ms	remaining: 2.96s
9:	learn: 2.8680104	total: 29.5ms	remaining: 2.92s
10:	learn: 2.8418908	total: 31.9ms	remaining: 2.87s
11:	learn: 2.8181057	total: 34.5ms	remaining: 2.84s
12:	learn: 2.7962279	total: 37ms	remaining: 2.81s
13:	learn: 2.7755928	total: 39.2ms	remaining: 2.76s
14:	learn: 2.7586136	total: 41.5ms	remaining: 2.72s
15:	learn: 2.7379041	total: 43.8ms	remaining: 2.69s
16:	learn: 2.7192303	total: 46.1ms	remaining: 2.66s
17:	learn: 2.7040307	total: 48.2ms	remaining: 2.63s
18:	learn: 2.6909298	total: 50.5ms	remaining: 2.61

260:	learn: 2.0239585	total: 388ms	remaining: 1.1s
261:	learn: 2.0226816	total: 389ms	remaining: 1.1s
262:	learn: 2.0216490	total: 391ms	remaining: 1.09s
263:	learn: 2.0198214	total: 392ms	remaining: 1.09s
264:	learn: 2.0189290	total: 393ms	remaining: 1.09s
265:	learn: 2.0178287	total: 395ms	remaining: 1.09s
266:	learn: 2.0163430	total: 396ms	remaining: 1.09s
267:	learn: 2.0153386	total: 397ms	remaining: 1.08s
268:	learn: 2.0141765	total: 399ms	remaining: 1.08s
269:	learn: 2.0133080	total: 400ms	remaining: 1.08s
270:	learn: 2.0115957	total: 401ms	remaining: 1.08s
271:	learn: 2.0070943	total: 402ms	remaining: 1.08s
272:	learn: 2.0063524	total: 404ms	remaining: 1.07s
273:	learn: 2.0037714	total: 405ms	remaining: 1.07s
274:	learn: 2.0025901	total: 406ms	remaining: 1.07s
275:	learn: 2.0016630	total: 408ms	remaining: 1.07s
276:	learn: 2.0010499	total: 409ms	remaining: 1.07s
277:	learn: 2.0003358	total: 410ms	remaining: 1.07s
278:	learn: 1.9996317	total: 412ms	remaining: 1.06s
279:	learn: 1.

557:	learn: 1.6595417	total: 777ms	remaining: 615ms
558:	learn: 1.6589595	total: 778ms	remaining: 614ms
559:	learn: 1.6577164	total: 780ms	remaining: 613ms
560:	learn: 1.6569187	total: 781ms	remaining: 611ms
561:	learn: 1.6564104	total: 782ms	remaining: 610ms
562:	learn: 1.6558431	total: 783ms	remaining: 608ms
563:	learn: 1.6553523	total: 785ms	remaining: 607ms
564:	learn: 1.6550689	total: 786ms	remaining: 605ms
565:	learn: 1.6534117	total: 787ms	remaining: 604ms
566:	learn: 1.6528560	total: 789ms	remaining: 602ms
567:	learn: 1.6523096	total: 790ms	remaining: 601ms
568:	learn: 1.6504182	total: 791ms	remaining: 599ms
569:	learn: 1.6497687	total: 793ms	remaining: 598ms
570:	learn: 1.6490137	total: 794ms	remaining: 596ms
571:	learn: 1.6481474	total: 795ms	remaining: 595ms
572:	learn: 1.6475817	total: 796ms	remaining: 593ms
573:	learn: 1.6470396	total: 798ms	remaining: 592ms
574:	learn: 1.6464499	total: 799ms	remaining: 591ms
575:	learn: 1.6448389	total: 800ms	remaining: 589ms
576:	learn: 

852:	learn: 1.4272296	total: 1.17s	remaining: 201ms
853:	learn: 1.4254094	total: 1.17s	remaining: 199ms
854:	learn: 1.4249683	total: 1.17s	remaining: 198ms
855:	learn: 1.4242102	total: 1.17s	remaining: 197ms
856:	learn: 1.4227239	total: 1.17s	remaining: 195ms
857:	learn: 1.4222147	total: 1.17s	remaining: 194ms
858:	learn: 1.4216998	total: 1.17s	remaining: 193ms
859:	learn: 1.4215249	total: 1.17s	remaining: 191ms
860:	learn: 1.4213443	total: 1.18s	remaining: 190ms
861:	learn: 1.4206435	total: 1.18s	remaining: 188ms
862:	learn: 1.4198230	total: 1.18s	remaining: 187ms
863:	learn: 1.4189901	total: 1.18s	remaining: 186ms
864:	learn: 1.4173208	total: 1.18s	remaining: 184ms
865:	learn: 1.4170368	total: 1.18s	remaining: 183ms
866:	learn: 1.4162193	total: 1.18s	remaining: 182ms
867:	learn: 1.4151709	total: 1.19s	remaining: 180ms
868:	learn: 1.4144024	total: 1.19s	remaining: 179ms
869:	learn: 1.4140037	total: 1.19s	remaining: 177ms
870:	learn: 1.4138370	total: 1.19s	remaining: 176ms
871:	learn: 

98:	learn: 2.1227277	total: 192ms	remaining: 1.75s
99:	learn: 2.1213695	total: 194ms	remaining: 1.74s
100:	learn: 2.1182097	total: 195ms	remaining: 1.73s
101:	learn: 2.1151553	total: 196ms	remaining: 1.73s
102:	learn: 2.1127888	total: 197ms	remaining: 1.72s
103:	learn: 2.1113796	total: 199ms	remaining: 1.71s
104:	learn: 2.1084051	total: 200ms	remaining: 1.7s
105:	learn: 2.1065321	total: 201ms	remaining: 1.7s
106:	learn: 2.1047459	total: 202ms	remaining: 1.69s
107:	learn: 2.1006363	total: 204ms	remaining: 1.68s
108:	learn: 2.0992564	total: 205ms	remaining: 1.68s
109:	learn: 2.0974147	total: 206ms	remaining: 1.67s
110:	learn: 2.0932357	total: 208ms	remaining: 1.66s
111:	learn: 2.0891939	total: 209ms	remaining: 1.66s
112:	learn: 2.0871260	total: 210ms	remaining: 1.65s
113:	learn: 2.0855266	total: 211ms	remaining: 1.64s
114:	learn: 2.0837317	total: 213ms	remaining: 1.64s
115:	learn: 2.0813176	total: 214ms	remaining: 1.63s
116:	learn: 2.0786959	total: 215ms	remaining: 1.62s
117:	learn: 2.07

...

622:	learn: 1.4028696	total: 878ms	remaining: 531ms
623:	learn: 1.4026467	total: 880ms	remaining: 530ms
624:	learn: 1.4012442	total: 881ms	remaining: 529ms
625:	learn: 1.4007022	total: 882ms	remaining: 527ms
626:	learn: 1.4002982	total: 883ms	remaining: 525ms
627:	learn: 1.4000719	total: 885ms	remaining: 524ms
628:	learn: 1.3993806	total: 886ms	remaining: 522ms
629:	learn: 1.3981110	total: 887ms	remaining: 521ms
630:	learn: 1.3975754	total: 888ms	remaining: 520ms
631:	learn: 1.3971123	total: 890ms	remaining: 518ms
632:	learn: 1.3956109	total: 891ms	remaining: 517ms
633:	learn: 1.3953850	total: 892ms	remaining: 515ms
634:	learn: 1.3949358	total: 894ms	remaining: 514ms
635:	learn: 1.3945416	total: 895ms	remaining: 512ms
636:	learn: 1.3943518	total: 896ms	remaining: 511ms
637:	learn: 1.3937286	total: 898ms	remaining: 509ms
638:	learn: 1.3930515	total: 899ms	remaining: 508ms
639:	learn: 1.3923461	total: 900ms	remaining: 506ms
640:	learn: 1.3910395	total: 902ms	remaining: 505ms
641:	learn: 

918:	learn: 1.2087106	total: 1.26s	remaining: 112ms
919:	learn: 1.2081523	total: 1.27s	remaining: 110ms
920:	learn: 1.2078854	total: 1.27s	remaining: 109ms
921:	learn: 1.2065376	total: 1.27s	remaining: 107ms
922:	learn: 1.2063172	total: 1.27s	remaining: 106ms
923:	learn: 1.2061729	total: 1.27s	remaining: 105ms
924:	learn: 1.2059329	total: 1.27s	remaining: 103ms
925:	learn: 1.2052128	total: 1.27s	remaining: 102ms
926:	learn: 1.2048913	total: 1.27s	remaining: 100ms
927:	learn: 1.2040097	total: 1.28s	remaining: 99.1ms
928:	learn: 1.2037654	total: 1.28s	remaining: 97.7ms
929:	learn: 1.2034716	total: 1.28s	remaining: 96.3ms
930:	learn: 1.2029565	total: 1.28s	remaining: 95ms
931:	learn: 1.2020092	total: 1.28s	remaining: 93.6ms
932:	learn: 1.2016930	total: 1.28s	remaining: 92.2ms
933:	learn: 1.2013533	total: 1.28s	remaining: 90.8ms
934:	learn: 1.2009088	total: 1.29s	remaining: 89.4ms
935:	learn: 1.2005270	total: 1.29s	remaining: 88ms
936:	learn: 1.1996221	total: 1.29s	remaining: 86.7ms
937:	l

106:	learn: 2.1083810	total: 194ms	remaining: 1.61s
107:	learn: 2.1056466	total: 195ms	remaining: 1.61s
108:	learn: 2.1035087	total: 196ms	remaining: 1.6s
109:	learn: 2.0994936	total: 198ms	remaining: 1.6s
110:	learn: 2.0980375	total: 199ms	remaining: 1.59s
111:	learn: 2.0965965	total: 200ms	remaining: 1.59s
112:	learn: 2.0957211	total: 201ms	remaining: 1.58s
113:	learn: 2.0938345	total: 203ms	remaining: 1.57s
114:	learn: 2.0927348	total: 204ms	remaining: 1.57s
115:	learn: 2.0913671	total: 205ms	remaining: 1.56s
116:	learn: 2.0901039	total: 206ms	remaining: 1.56s
117:	learn: 2.0893754	total: 208ms	remaining: 1.55s
118:	learn: 2.0874009	total: 209ms	remaining: 1.55s
119:	learn: 2.0865703	total: 210ms	remaining: 1.54s
120:	learn: 2.0840294	total: 211ms	remaining: 1.53s
121:	learn: 2.0823384	total: 213ms	remaining: 1.53s
122:	learn: 2.0797616	total: 214ms	remaining: 1.53s
123:	learn: 2.0781523	total: 215ms	remaining: 1.52s
124:	learn: 2.0757750	total: 217ms	remaining: 1.52s
125:	learn: 2.

408:	learn: 1.6782040	total: 581ms	remaining: 839ms
409:	learn: 1.6774867	total: 582ms	remaining: 838ms
410:	learn: 1.6765500	total: 583ms	remaining: 836ms
411:	learn: 1.6756842	total: 585ms	remaining: 834ms
412:	learn: 1.6750566	total: 586ms	remaining: 832ms
413:	learn: 1.6744236	total: 587ms	remaining: 831ms
414:	learn: 1.6739070	total: 588ms	remaining: 829ms
415:	learn: 1.6727539	total: 589ms	remaining: 828ms
416:	learn: 1.6721010	total: 591ms	remaining: 826ms
417:	learn: 1.6714662	total: 592ms	remaining: 824ms
418:	learn: 1.6711702	total: 593ms	remaining: 823ms
419:	learn: 1.6706568	total: 594ms	remaining: 821ms
420:	learn: 1.6697976	total: 596ms	remaining: 819ms
421:	learn: 1.6686192	total: 597ms	remaining: 818ms
422:	learn: 1.6667749	total: 598ms	remaining: 816ms
423:	learn: 1.6661889	total: 600ms	remaining: 815ms
424:	learn: 1.6658993	total: 601ms	remaining: 813ms
425:	learn: 1.6652334	total: 602ms	remaining: 811ms
426:	learn: 1.6643757	total: 603ms	remaining: 810ms
427:	learn: 

706:	learn: 1.4106511	total: 968ms	remaining: 401ms
707:	learn: 1.4102792	total: 970ms	remaining: 400ms
708:	learn: 1.4096491	total: 971ms	remaining: 399ms
709:	learn: 1.4091396	total: 972ms	remaining: 397ms
710:	learn: 1.4087325	total: 974ms	remaining: 396ms
711:	learn: 1.4083566	total: 975ms	remaining: 394ms
712:	learn: 1.4077534	total: 976ms	remaining: 393ms
713:	learn: 1.4075759	total: 977ms	remaining: 392ms
714:	learn: 1.4071752	total: 979ms	remaining: 390ms
715:	learn: 1.4061611	total: 980ms	remaining: 389ms
716:	learn: 1.4056540	total: 981ms	remaining: 387ms
717:	learn: 1.4053216	total: 983ms	remaining: 386ms
718:	learn: 1.4049139	total: 984ms	remaining: 384ms
719:	learn: 1.4043710	total: 985ms	remaining: 383ms
720:	learn: 1.4038351	total: 986ms	remaining: 382ms
721:	learn: 1.4034213	total: 988ms	remaining: 380ms
722:	learn: 1.4029418	total: 989ms	remaining: 379ms
723:	learn: 1.4025482	total: 990ms	remaining: 377ms
724:	learn: 1.4023470	total: 991ms	remaining: 376ms
725:	learn: 

In [12]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.1231684626152818
1.1199747087957155
1.1141167921908097
1.1251186580223131
1.1208220801800095
1.135810073887561
0.49678364473028436
0.5092716673436332
0.4813149102330311
0.519138171218313
0.47019916242540094
0.5039943124310435


In [13]:
train_size = 4000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.050969
0:	learn: 3.2133510	total: 4.93ms	remaining: 4.93s
1:	learn: 3.1668132	total: 9.34ms	remaining: 4.66s
2:	learn: 3.1193531	total: 13.6ms	remaining: 4.51s
3:	learn: 3.0773790	total: 17.1ms	remaining: 4.26s
4:	learn: 3.0383843	total: 20.9ms	remaining: 4.15s
5:	learn: 3.0022037	total: 24.4ms	remaining: 4.04s
6:	learn: 2.9713169	total: 27.4ms	remaining: 3.88s
7:	learn: 2.9387782	total: 30.4ms	remaining: 3.77s
8:	learn: 2.9072048	total: 33.7ms	remaining: 3.71s
9:	learn: 2.8812490	total: 36.9ms	remaining: 3.65s
10:	learn: 2.8554619	total: 40ms	remaining: 3.59s
11:	learn: 2.8322825	total: 43ms	remaining: 3.54s
12:	learn: 2.8106021	total: 46.1ms	remaining: 3.5s
13:	learn: 2.7910359	total: 49.4ms	remaining: 3.48s
14:	learn: 2.7730191	total: 52.4ms	remaining: 3.44s
15:	learn: 2.7558167	total: 55.6ms	remaining: 3.42s
16:	learn: 2.7391596	total: 58.6ms	remaining: 3.39s
17:	learn: 2.7224633	total: 61.3ms	remaining: 3.34s
18:	learn: 2.7070721	total: 64.3ms	remaining: 3.3

297:	learn: 2.0117892	total: 586ms	remaining: 1.38s
298:	learn: 2.0096249	total: 587ms	remaining: 1.38s
299:	learn: 2.0084092	total: 589ms	remaining: 1.37s
300:	learn: 2.0073602	total: 590ms	remaining: 1.37s
301:	learn: 2.0054506	total: 591ms	remaining: 1.37s
302:	learn: 2.0038855	total: 593ms	remaining: 1.36s
303:	learn: 2.0021834	total: 594ms	remaining: 1.36s
304:	learn: 2.0013561	total: 596ms	remaining: 1.36s
305:	learn: 1.9997299	total: 597ms	remaining: 1.35s
306:	learn: 1.9975131	total: 598ms	remaining: 1.35s
307:	learn: 1.9968461	total: 600ms	remaining: 1.35s
308:	learn: 1.9954386	total: 601ms	remaining: 1.34s
309:	learn: 1.9934813	total: 602ms	remaining: 1.34s
310:	learn: 1.9928995	total: 604ms	remaining: 1.34s
311:	learn: 1.9917101	total: 605ms	remaining: 1.33s
312:	learn: 1.9908396	total: 607ms	remaining: 1.33s
313:	learn: 1.9894453	total: 608ms	remaining: 1.33s
314:	learn: 1.9887430	total: 609ms	remaining: 1.32s
315:	learn: 1.9882735	total: 611ms	remaining: 1.32s
316:	learn: 

581:	learn: 1.7148298	total: 975ms	remaining: 700ms
582:	learn: 1.7143358	total: 977ms	remaining: 698ms
583:	learn: 1.7131045	total: 978ms	remaining: 697ms
584:	learn: 1.7116782	total: 979ms	remaining: 695ms
585:	learn: 1.7114511	total: 981ms	remaining: 693ms
586:	learn: 1.7110536	total: 982ms	remaining: 691ms
587:	learn: 1.7106282	total: 983ms	remaining: 689ms
588:	learn: 1.7099252	total: 985ms	remaining: 687ms
589:	learn: 1.7092590	total: 986ms	remaining: 685ms
590:	learn: 1.7088678	total: 987ms	remaining: 683ms
591:	learn: 1.7086659	total: 989ms	remaining: 681ms
592:	learn: 1.7074336	total: 990ms	remaining: 680ms
593:	learn: 1.7072522	total: 992ms	remaining: 678ms
594:	learn: 1.7054177	total: 993ms	remaining: 676ms
595:	learn: 1.7050570	total: 994ms	remaining: 674ms
596:	learn: 1.7037658	total: 996ms	remaining: 672ms
597:	learn: 1.7021383	total: 997ms	remaining: 670ms
598:	learn: 1.7019138	total: 998ms	remaining: 668ms
599:	learn: 1.7011745	total: 1000ms	remaining: 667ms
600:	learn:

865:	learn: 1.5073585	total: 1.36s	remaining: 211ms
866:	learn: 1.5072262	total: 1.37s	remaining: 210ms
867:	learn: 1.5052301	total: 1.37s	remaining: 208ms
868:	learn: 1.5043649	total: 1.37s	remaining: 206ms
869:	learn: 1.5038207	total: 1.37s	remaining: 205ms
870:	learn: 1.5032030	total: 1.37s	remaining: 203ms
871:	learn: 1.5027748	total: 1.37s	remaining: 202ms
872:	learn: 1.5026514	total: 1.37s	remaining: 200ms
873:	learn: 1.5015933	total: 1.38s	remaining: 198ms
874:	learn: 1.4998461	total: 1.38s	remaining: 197ms
875:	learn: 1.4986380	total: 1.38s	remaining: 195ms
876:	learn: 1.4983302	total: 1.38s	remaining: 194ms
877:	learn: 1.4980840	total: 1.38s	remaining: 192ms
878:	learn: 1.4976730	total: 1.38s	remaining: 190ms
879:	learn: 1.4964662	total: 1.38s	remaining: 189ms
880:	learn: 1.4960562	total: 1.39s	remaining: 187ms
881:	learn: 1.4957900	total: 1.39s	remaining: 186ms
882:	learn: 1.4949030	total: 1.39s	remaining: 184ms
883:	learn: 1.4940409	total: 1.39s	remaining: 182ms
884:	learn: 

88:	learn: 2.1341514	total: 195ms	remaining: 1.99s
89:	learn: 2.1314137	total: 196ms	remaining: 1.98s
90:	learn: 2.1287419	total: 198ms	remaining: 1.97s
91:	learn: 2.1244535	total: 199ms	remaining: 1.96s
92:	learn: 2.1221144	total: 200ms	remaining: 1.95s
93:	learn: 2.1204324	total: 202ms	remaining: 1.94s
94:	learn: 2.1187360	total: 203ms	remaining: 1.94s
95:	learn: 2.1156371	total: 205ms	remaining: 1.93s
96:	learn: 2.1122629	total: 206ms	remaining: 1.92s
97:	learn: 2.1088491	total: 207ms	remaining: 1.91s
98:	learn: 2.1042001	total: 209ms	remaining: 1.9s
99:	learn: 2.1022273	total: 210ms	remaining: 1.89s
100:	learn: 2.0989366	total: 211ms	remaining: 1.88s
101:	learn: 2.0960362	total: 213ms	remaining: 1.87s
102:	learn: 2.0928363	total: 214ms	remaining: 1.86s
103:	learn: 2.0910292	total: 216ms	remaining: 1.86s
104:	learn: 2.0896163	total: 217ms	remaining: 1.85s
105:	learn: 2.0861418	total: 218ms	remaining: 1.84s
106:	learn: 2.0847094	total: 220ms	remaining: 1.83s
107:	learn: 2.0825271	tot

...

765:	learn: 1.4242955	total: 1.19s	remaining: 362ms
766:	learn: 1.4238815	total: 1.19s	remaining: 361ms
767:	learn: 1.4230605	total: 1.19s	remaining: 359ms
768:	learn: 1.4226289	total: 1.19s	remaining: 358ms
769:	learn: 1.4209378	total: 1.19s	remaining: 356ms
770:	learn: 1.4202944	total: 1.19s	remaining: 355ms
771:	learn: 1.4190573	total: 1.2s	remaining: 353ms
772:	learn: 1.4188746	total: 1.2s	remaining: 351ms
773:	learn: 1.4172287	total: 1.2s	remaining: 350ms
774:	learn: 1.4170480	total: 1.2s	remaining: 348ms
775:	learn: 1.4164452	total: 1.2s	remaining: 347ms
776:	learn: 1.4159531	total: 1.2s	remaining: 345ms
777:	learn: 1.4157741	total: 1.2s	remaining: 343ms
778:	learn: 1.4148700	total: 1.2s	remaining: 342ms
779:	learn: 1.4143181	total: 1.21s	remaining: 340ms
780:	learn: 1.4141418	total: 1.21s	remaining: 339ms
781:	learn: 1.4135032	total: 1.21s	remaining: 337ms
782:	learn: 1.4129963	total: 1.21s	remaining: 335ms
783:	learn: 1.4124851	total: 1.21s	remaining: 334ms
784:	learn: 1.411983

Learning rate set to 0.050969
0:	learn: 3.0981866	total: 3.85ms	remaining: 3.85s
1:	learn: 3.0441704	total: 7.54ms	remaining: 3.76s
2:	learn: 2.9942348	total: 11ms	remaining: 3.66s
3:	learn: 2.9484248	total: 14.2ms	remaining: 3.54s
4:	learn: 2.9080908	total: 17.4ms	remaining: 3.47s
5:	learn: 2.8703304	total: 23.2ms	remaining: 3.85s
6:	learn: 2.8298235	total: 26.4ms	remaining: 3.74s
7:	learn: 2.7942461	total: 29.6ms	remaining: 3.67s
8:	learn: 2.7592794	total: 32.8ms	remaining: 3.61s
9:	learn: 2.7295401	total: 35.8ms	remaining: 3.54s
10:	learn: 2.7020216	total: 38.5ms	remaining: 3.46s
11:	learn: 2.6725385	total: 41.5ms	remaining: 3.42s
12:	learn: 2.6448277	total: 44.6ms	remaining: 3.39s
13:	learn: 2.6194921	total: 47.7ms	remaining: 3.36s
14:	learn: 2.5975503	total: 50.7ms	remaining: 3.33s
15:	learn: 2.5756152	total: 53.6ms	remaining: 3.29s
16:	learn: 2.5546034	total: 56.6ms	remaining: 3.27s
17:	learn: 2.5383004	total: 59.3ms	remaining: 3.23s
18:	learn: 2.5217823	total: 62.1ms	remaining: 

174:	learn: 1.9637177	total: 389ms	remaining: 1.83s
175:	learn: 1.9630110	total: 391ms	remaining: 1.83s
176:	learn: 1.9621072	total: 392ms	remaining: 1.82s
177:	learn: 1.9609952	total: 393ms	remaining: 1.82s
178:	learn: 1.9591196	total: 395ms	remaining: 1.81s
179:	learn: 1.9573543	total: 396ms	remaining: 1.8s
180:	learn: 1.9550335	total: 398ms	remaining: 1.8s
181:	learn: 1.9519225	total: 399ms	remaining: 1.79s
182:	learn: 1.9509538	total: 400ms	remaining: 1.79s
183:	learn: 1.9500878	total: 402ms	remaining: 1.78s
184:	learn: 1.9494262	total: 403ms	remaining: 1.77s
185:	learn: 1.9484627	total: 404ms	remaining: 1.77s
186:	learn: 1.9473324	total: 406ms	remaining: 1.76s
187:	learn: 1.9456814	total: 407ms	remaining: 1.76s
188:	learn: 1.9442537	total: 409ms	remaining: 1.75s
189:	learn: 1.9419449	total: 410ms	remaining: 1.75s
190:	learn: 1.9396442	total: 411ms	remaining: 1.74s
191:	learn: 1.9389476	total: 413ms	remaining: 1.74s
192:	learn: 1.9379707	total: 414ms	remaining: 1.73s
193:	learn: 1.

456:	learn: 1.6301724	total: 779ms	remaining: 926ms
457:	learn: 1.6296198	total: 781ms	remaining: 924ms
458:	learn: 1.6277560	total: 782ms	remaining: 922ms
459:	learn: 1.6271190	total: 783ms	remaining: 920ms
460:	learn: 1.6264849	total: 785ms	remaining: 917ms
461:	learn: 1.6260518	total: 786ms	remaining: 915ms
462:	learn: 1.6245754	total: 787ms	remaining: 913ms
463:	learn: 1.6243132	total: 789ms	remaining: 911ms
464:	learn: 1.6230135	total: 790ms	remaining: 909ms
465:	learn: 1.6214556	total: 791ms	remaining: 907ms
466:	learn: 1.6211990	total: 793ms	remaining: 905ms
467:	learn: 1.6209547	total: 794ms	remaining: 903ms
468:	learn: 1.6192026	total: 796ms	remaining: 901ms
469:	learn: 1.6182797	total: 797ms	remaining: 899ms
470:	learn: 1.6176791	total: 798ms	remaining: 897ms
471:	learn: 1.6158524	total: 800ms	remaining: 895ms
472:	learn: 1.6152900	total: 801ms	remaining: 893ms
473:	learn: 1.6146966	total: 803ms	remaining: 891ms
474:	learn: 1.6138501	total: 804ms	remaining: 889ms
475:	learn: 

736:	learn: 1.4269398	total: 1.17s	remaining: 417ms
737:	learn: 1.4265340	total: 1.17s	remaining: 416ms
738:	learn: 1.4251822	total: 1.17s	remaining: 414ms
739:	learn: 1.4247231	total: 1.17s	remaining: 412ms
740:	learn: 1.4245582	total: 1.17s	remaining: 411ms
741:	learn: 1.4238944	total: 1.18s	remaining: 409ms
742:	learn: 1.4233395	total: 1.18s	remaining: 407ms
743:	learn: 1.4227237	total: 1.18s	remaining: 406ms
744:	learn: 1.4216772	total: 1.18s	remaining: 404ms
745:	learn: 1.4214460	total: 1.18s	remaining: 402ms
746:	learn: 1.4209688	total: 1.18s	remaining: 401ms
747:	learn: 1.4197585	total: 1.18s	remaining: 399ms
748:	learn: 1.4195351	total: 1.19s	remaining: 397ms
749:	learn: 1.4182906	total: 1.19s	remaining: 396ms
750:	learn: 1.4177507	total: 1.19s	remaining: 394ms
751:	learn: 1.4175285	total: 1.19s	remaining: 392ms
752:	learn: 1.4170726	total: 1.19s	remaining: 391ms
753:	learn: 1.4160878	total: 1.19s	remaining: 389ms
754:	learn: 1.4156848	total: 1.19s	remaining: 387ms
755:	learn: 

In [14]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.108246216943774
1.0825861533233383
1.0834027468627523
1.151542810962691
1.1099409243614384
1.1137584492086485
0.5184853630382121
0.5451700200108103
0.5008271747370581
0.5100207957106836
0.4980827064686888
0.5383261182638202


In [15]:
train_size = 5000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.052798
0:	learn: 3.1469239	total: 3.81ms	remaining: 3.8s
1:	learn: 3.0998016	total: 8.14ms	remaining: 4.06s
2:	learn: 3.0541601	total: 12ms	remaining: 4s
3:	learn: 3.0085823	total: 15.7ms	remaining: 3.9s
4:	learn: 2.9661289	total: 19.4ms	remaining: 3.86s
5:	learn: 2.9295821	total: 22.7ms	remaining: 3.77s
6:	learn: 2.8944342	total: 26ms	remaining: 3.69s
7:	learn: 2.8632080	total: 29.5ms	remaining: 3.66s
8:	learn: 2.8344114	total: 32.7ms	remaining: 3.6s
9:	learn: 2.8053003	total: 36ms	remaining: 3.56s
10:	learn: 2.7802728	total: 39.4ms	remaining: 3.54s
11:	learn: 2.7555311	total: 42.8ms	remaining: 3.52s
12:	learn: 2.7328003	total: 46ms	remaining: 3.49s
13:	learn: 2.7114825	total: 49ms	remaining: 3.45s
14:	learn: 2.6904150	total: 51.9ms	remaining: 3.41s
15:	learn: 2.6726091	total: 54.8ms	remaining: 3.37s
16:	learn: 2.6571108	total: 57.6ms	remaining: 3.33s
17:	learn: 2.6437972	total: 60.5ms	remaining: 3.3s
18:	learn: 2.6303296	total: 63.4ms	remaining: 3.27s
19:	learn

191:	learn: 2.1088234	total: 593ms	remaining: 2.49s
192:	learn: 2.1073207	total: 596ms	remaining: 2.49s
193:	learn: 2.1044517	total: 599ms	remaining: 2.49s
194:	learn: 2.1026364	total: 602ms	remaining: 2.48s
195:	learn: 2.1010820	total: 605ms	remaining: 2.48s
196:	learn: 2.0987500	total: 608ms	remaining: 2.48s
197:	learn: 2.0966951	total: 611ms	remaining: 2.47s
198:	learn: 2.0945318	total: 614ms	remaining: 2.47s
199:	learn: 2.0936132	total: 617ms	remaining: 2.47s
200:	learn: 2.0922756	total: 620ms	remaining: 2.46s
201:	learn: 2.0900160	total: 623ms	remaining: 2.46s
202:	learn: 2.0882633	total: 627ms	remaining: 2.46s
203:	learn: 2.0864643	total: 630ms	remaining: 2.46s
204:	learn: 2.0847819	total: 633ms	remaining: 2.45s
205:	learn: 2.0832404	total: 635ms	remaining: 2.45s
206:	learn: 2.0826161	total: 638ms	remaining: 2.44s
207:	learn: 2.0819583	total: 642ms	remaining: 2.44s
208:	learn: 2.0804524	total: 645ms	remaining: 2.44s
209:	learn: 2.0798736	total: 648ms	remaining: 2.44s
210:	learn: 

395:	learn: 1.8789376	total: 1.19s	remaining: 1.81s
396:	learn: 1.8766042	total: 1.19s	remaining: 1.81s
397:	learn: 1.8762648	total: 1.2s	remaining: 1.81s
398:	learn: 1.8757696	total: 1.2s	remaining: 1.8s
399:	learn: 1.8747929	total: 1.2s	remaining: 1.8s
400:	learn: 1.8735408	total: 1.2s	remaining: 1.8s
401:	learn: 1.8727594	total: 1.21s	remaining: 1.8s
402:	learn: 1.8721411	total: 1.21s	remaining: 1.79s
403:	learn: 1.8718073	total: 1.21s	remaining: 1.79s
404:	learn: 1.8699200	total: 1.22s	remaining: 1.79s
405:	learn: 1.8667163	total: 1.22s	remaining: 1.78s
406:	learn: 1.8661910	total: 1.22s	remaining: 1.78s
407:	learn: 1.8646482	total: 1.23s	remaining: 1.78s
408:	learn: 1.8638178	total: 1.23s	remaining: 1.78s
409:	learn: 1.8632962	total: 1.23s	remaining: 1.77s
410:	learn: 1.8628246	total: 1.24s	remaining: 1.77s
411:	learn: 1.8624158	total: 1.24s	remaining: 1.77s
412:	learn: 1.8605783	total: 1.24s	remaining: 1.76s
413:	learn: 1.8599766	total: 1.24s	remaining: 1.76s
414:	learn: 1.859637

619:	learn: 1.6908776	total: 1.78s	remaining: 1.09s
620:	learn: 1.6904161	total: 1.78s	remaining: 1.09s
621:	learn: 1.6891038	total: 1.78s	remaining: 1.08s
622:	learn: 1.6878925	total: 1.79s	remaining: 1.08s
623:	learn: 1.6869809	total: 1.79s	remaining: 1.08s
624:	learn: 1.6861410	total: 1.79s	remaining: 1.07s
625:	learn: 1.6855769	total: 1.79s	remaining: 1.07s
626:	learn: 1.6843615	total: 1.79s	remaining: 1.07s
627:	learn: 1.6825287	total: 1.79s	remaining: 1.06s
628:	learn: 1.6823024	total: 1.8s	remaining: 1.06s
629:	learn: 1.6819015	total: 1.8s	remaining: 1.05s
630:	learn: 1.6811873	total: 1.8s	remaining: 1.05s
631:	learn: 1.6806796	total: 1.8s	remaining: 1.05s
632:	learn: 1.6802223	total: 1.8s	remaining: 1.04s
633:	learn: 1.6793561	total: 1.8s	remaining: 1.04s
634:	learn: 1.6779702	total: 1.8s	remaining: 1.04s
635:	learn: 1.6760900	total: 1.81s	remaining: 1.03s
636:	learn: 1.6759127	total: 1.81s	remaining: 1.03s
637:	learn: 1.6753370	total: 1.81s	remaining: 1.03s
638:	learn: 1.67488

883:	learn: 1.5086329	total: 2.17s	remaining: 285ms
884:	learn: 1.5083245	total: 2.17s	remaining: 283ms
885:	learn: 1.5080305	total: 2.18s	remaining: 280ms
886:	learn: 1.5078592	total: 2.18s	remaining: 278ms
887:	learn: 1.5075761	total: 2.18s	remaining: 275ms
888:	learn: 1.5066387	total: 2.18s	remaining: 272ms
889:	learn: 1.5064763	total: 2.18s	remaining: 270ms
890:	learn: 1.5062195	total: 2.18s	remaining: 267ms
891:	learn: 1.5058829	total: 2.19s	remaining: 265ms
892:	learn: 1.5046842	total: 2.19s	remaining: 262ms
893:	learn: 1.5045182	total: 2.19s	remaining: 260ms
894:	learn: 1.5042468	total: 2.19s	remaining: 257ms
895:	learn: 1.5038663	total: 2.19s	remaining: 254ms
896:	learn: 1.5025719	total: 2.19s	remaining: 252ms
897:	learn: 1.5011771	total: 2.19s	remaining: 249ms
898:	learn: 1.5010325	total: 2.2s	remaining: 247ms
899:	learn: 1.4995139	total: 2.2s	remaining: 244ms
900:	learn: 1.4985954	total: 2.2s	remaining: 242ms
901:	learn: 1.4978328	total: 2.2s	remaining: 239ms
902:	learn: 1.49

...

175:	learn: 2.0207201	total: 590ms	remaining: 2.76s
176:	learn: 2.0199542	total: 595ms	remaining: 2.77s
177:	learn: 2.0166379	total: 599ms	remaining: 2.77s
178:	learn: 2.0157716	total: 603ms	remaining: 2.77s
179:	learn: 2.0131786	total: 607ms	remaining: 2.77s
180:	learn: 2.0123868	total: 611ms	remaining: 2.77s
181:	learn: 2.0102469	total: 615ms	remaining: 2.76s
182:	learn: 2.0094979	total: 618ms	remaining: 2.76s
183:	learn: 2.0076256	total: 621ms	remaining: 2.75s
184:	learn: 2.0064052	total: 624ms	remaining: 2.75s
185:	learn: 2.0040315	total: 627ms	remaining: 2.74s
186:	learn: 2.0011752	total: 630ms	remaining: 2.74s
187:	learn: 2.0002751	total: 633ms	remaining: 2.73s
188:	learn: 1.9985926	total: 636ms	remaining: 2.73s
189:	learn: 1.9966017	total: 638ms	remaining: 2.72s
190:	learn: 1.9957723	total: 641ms	remaining: 2.72s
191:	learn: 1.9944996	total: 644ms	remaining: 2.71s
192:	learn: 1.9936817	total: 647ms	remaining: 2.71s
193:	learn: 1.9921192	total: 650ms	remaining: 2.7s
194:	learn: 1

361:	learn: 1.7830321	total: 1.18s	remaining: 2.09s
362:	learn: 1.7821590	total: 1.19s	remaining: 2.08s
363:	learn: 1.7817332	total: 1.19s	remaining: 2.08s
364:	learn: 1.7800597	total: 1.19s	remaining: 2.08s
365:	learn: 1.7792670	total: 1.2s	remaining: 2.07s
366:	learn: 1.7780005	total: 1.2s	remaining: 2.07s
367:	learn: 1.7772393	total: 1.2s	remaining: 2.06s
368:	learn: 1.7753376	total: 1.21s	remaining: 2.06s
369:	learn: 1.7745300	total: 1.21s	remaining: 2.06s
370:	learn: 1.7731466	total: 1.22s	remaining: 2.06s
371:	learn: 1.7722818	total: 1.22s	remaining: 2.06s
372:	learn: 1.7718752	total: 1.22s	remaining: 2.06s
373:	learn: 1.7708909	total: 1.23s	remaining: 2.05s
374:	learn: 1.7702604	total: 1.23s	remaining: 2.05s
375:	learn: 1.7687507	total: 1.23s	remaining: 2.05s
376:	learn: 1.7679904	total: 1.24s	remaining: 2.04s
377:	learn: 1.7667585	total: 1.24s	remaining: 2.04s
378:	learn: 1.7652167	total: 1.24s	remaining: 2.04s
379:	learn: 1.7622828	total: 1.25s	remaining: 2.04s
380:	learn: 1.7

538:	learn: 1.6248221	total: 1.78s	remaining: 1.52s
539:	learn: 1.6236735	total: 1.78s	remaining: 1.52s
540:	learn: 1.6222586	total: 1.79s	remaining: 1.52s
541:	learn: 1.6219049	total: 1.79s	remaining: 1.51s
542:	learn: 1.6215612	total: 1.79s	remaining: 1.51s
543:	learn: 1.6213961	total: 1.8s	remaining: 1.51s
544:	learn: 1.6205760	total: 1.8s	remaining: 1.5s
545:	learn: 1.6194365	total: 1.8s	remaining: 1.5s
546:	learn: 1.6188231	total: 1.81s	remaining: 1.5s
547:	learn: 1.6182990	total: 1.81s	remaining: 1.49s
548:	learn: 1.6164357	total: 1.81s	remaining: 1.49s
549:	learn: 1.6155331	total: 1.82s	remaining: 1.49s
550:	learn: 1.6152021	total: 1.82s	remaining: 1.48s
551:	learn: 1.6139006	total: 1.82s	remaining: 1.48s
552:	learn: 1.6128310	total: 1.83s	remaining: 1.48s
553:	learn: 1.6118897	total: 1.83s	remaining: 1.47s
554:	learn: 1.6112379	total: 1.83s	remaining: 1.47s
555:	learn: 1.6105478	total: 1.83s	remaining: 1.47s
556:	learn: 1.6088196	total: 1.84s	remaining: 1.46s
557:	learn: 1.6085

722:	learn: 1.4931583	total: 2.38s	remaining: 912ms
723:	learn: 1.4928604	total: 2.38s	remaining: 908ms
724:	learn: 1.4921891	total: 2.38s	remaining: 905ms
725:	learn: 1.4920431	total: 2.39s	remaining: 902ms
726:	learn: 1.4918067	total: 2.39s	remaining: 898ms
727:	learn: 1.4913176	total: 2.4s	remaining: 895ms
728:	learn: 1.4910789	total: 2.4s	remaining: 892ms
729:	learn: 1.4907653	total: 2.4s	remaining: 888ms
730:	learn: 1.4900907	total: 2.4s	remaining: 885ms
731:	learn: 1.4897434	total: 2.41s	remaining: 882ms
732:	learn: 1.4892028	total: 2.41s	remaining: 878ms
733:	learn: 1.4883123	total: 2.41s	remaining: 875ms
734:	learn: 1.4876985	total: 2.42s	remaining: 872ms
735:	learn: 1.4863589	total: 2.42s	remaining: 868ms
736:	learn: 1.4859916	total: 2.42s	remaining: 865ms
737:	learn: 1.4858586	total: 2.43s	remaining: 861ms
738:	learn: 1.4855284	total: 2.43s	remaining: 858ms
739:	learn: 1.4850680	total: 2.43s	remaining: 855ms
740:	learn: 1.4845453	total: 2.44s	remaining: 851ms
741:	learn: 1.48

910:	learn: 1.3903183	total: 2.98s	remaining: 291ms
911:	learn: 1.3899818	total: 2.98s	remaining: 287ms
912:	learn: 1.3891398	total: 2.98s	remaining: 284ms
913:	learn: 1.3890327	total: 2.98s	remaining: 281ms
914:	learn: 1.3887956	total: 2.99s	remaining: 278ms
915:	learn: 1.3884284	total: 2.99s	remaining: 274ms
916:	learn: 1.3883236	total: 2.99s	remaining: 271ms
917:	learn: 1.3878932	total: 3s	remaining: 268ms
918:	learn: 1.3877174	total: 3s	remaining: 264ms
919:	learn: 1.3874725	total: 3s	remaining: 261ms
920:	learn: 1.3867807	total: 3s	remaining: 258ms
921:	learn: 1.3865505	total: 3.01s	remaining: 255ms
922:	learn: 1.3859680	total: 3.01s	remaining: 251ms
923:	learn: 1.3853442	total: 3.01s	remaining: 248ms
924:	learn: 1.3845137	total: 3.02s	remaining: 245ms
925:	learn: 1.3844098	total: 3.02s	remaining: 241ms
926:	learn: 1.3842011	total: 3.02s	remaining: 238ms
927:	learn: 1.3837945	total: 3.03s	remaining: 235ms
928:	learn: 1.3827538	total: 3.03s	remaining: 232ms
929:	learn: 1.3824917	to

In [16]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.090626337094387
1.0745144530453452
1.084454970827732
1.118963769587694
1.082083552188955
1.093114939822209
0.5385626623524461
0.5588235110005609
0.5180781902962595
0.5471435052172513
0.5373702391814432
0.5313978660667154


In [17]:
train_size = 6000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.054341
0:	learn: 3.1686290	total: 2.67ms	remaining: 2.67s
1:	learn: 3.1108652	total: 5.48ms	remaining: 2.73s
2:	learn: 3.0606859	total: 8.05ms	remaining: 2.67s
3:	learn: 3.0156205	total: 11.2ms	remaining: 2.79s
4:	learn: 2.9733092	total: 14.2ms	remaining: 2.83s
5:	learn: 2.9364656	total: 17.3ms	remaining: 2.86s
6:	learn: 2.9003151	total: 20.3ms	remaining: 2.88s
7:	learn: 2.8637275	total: 23.5ms	remaining: 2.92s
8:	learn: 2.8303039	total: 26.6ms	remaining: 2.93s
9:	learn: 2.8024404	total: 29.9ms	remaining: 2.96s
10:	learn: 2.7772244	total: 33.2ms	remaining: 2.98s
11:	learn: 2.7518577	total: 36.7ms	remaining: 3.02s
12:	learn: 2.7275889	total: 40.1ms	remaining: 3.04s
13:	learn: 2.7073260	total: 43.4ms	remaining: 3.06s
14:	learn: 2.6888665	total: 46.7ms	remaining: 3.07s
15:	learn: 2.6722172	total: 50ms	remaining: 3.08s
16:	learn: 2.6556097	total: 55ms	remaining: 3.18s
17:	learn: 2.6387842	total: 58.4ms	remaining: 3.19s
18:	learn: 2.6243425	total: 61.8ms	remaining: 3.

180:	learn: 2.1086767	total: 594ms	remaining: 2.69s
181:	learn: 2.1070677	total: 598ms	remaining: 2.69s
182:	learn: 2.1051751	total: 600ms	remaining: 2.68s
183:	learn: 2.1030468	total: 604ms	remaining: 2.68s
184:	learn: 2.1004899	total: 607ms	remaining: 2.67s
185:	learn: 2.0988484	total: 610ms	remaining: 2.67s
186:	learn: 2.0974892	total: 613ms	remaining: 2.67s
187:	learn: 2.0952083	total: 616ms	remaining: 2.66s
188:	learn: 2.0925129	total: 620ms	remaining: 2.66s
189:	learn: 2.0917554	total: 623ms	remaining: 2.65s
190:	learn: 2.0897762	total: 626ms	remaining: 2.65s
191:	learn: 2.0889305	total: 629ms	remaining: 2.65s
192:	learn: 2.0872487	total: 632ms	remaining: 2.64s
193:	learn: 2.0859460	total: 635ms	remaining: 2.64s
194:	learn: 2.0848614	total: 638ms	remaining: 2.63s
195:	learn: 2.0832331	total: 642ms	remaining: 2.63s
196:	learn: 2.0808168	total: 645ms	remaining: 2.63s
197:	learn: 2.0801771	total: 648ms	remaining: 2.63s
198:	learn: 2.0785835	total: 652ms	remaining: 2.62s
199:	learn: 

355:	learn: 1.8930949	total: 1.19s	remaining: 2.14s
356:	learn: 1.8920204	total: 1.19s	remaining: 2.14s
357:	learn: 1.8907282	total: 1.19s	remaining: 2.14s
358:	learn: 1.8900677	total: 1.2s	remaining: 2.13s
359:	learn: 1.8886324	total: 1.2s	remaining: 2.13s
360:	learn: 1.8871066	total: 1.2s	remaining: 2.13s
361:	learn: 1.8856842	total: 1.2s	remaining: 2.12s
362:	learn: 1.8848637	total: 1.21s	remaining: 2.12s
363:	learn: 1.8841315	total: 1.21s	remaining: 2.12s
364:	learn: 1.8816649	total: 1.21s	remaining: 2.11s
365:	learn: 1.8810008	total: 1.22s	remaining: 2.11s
366:	learn: 1.8807420	total: 1.22s	remaining: 2.1s
367:	learn: 1.8802345	total: 1.22s	remaining: 2.1s
368:	learn: 1.8778976	total: 1.23s	remaining: 2.1s
369:	learn: 1.8771364	total: 1.23s	remaining: 2.09s
370:	learn: 1.8765540	total: 1.23s	remaining: 2.09s
371:	learn: 1.8759939	total: 1.24s	remaining: 2.09s
372:	learn: 1.8750706	total: 1.24s	remaining: 2.09s
373:	learn: 1.8748174	total: 1.25s	remaining: 2.09s
374:	learn: 1.87394

541:	learn: 1.7245647	total: 1.78s	remaining: 1.51s
542:	learn: 1.7240795	total: 1.79s	remaining: 1.5s
543:	learn: 1.7236232	total: 1.79s	remaining: 1.5s
544:	learn: 1.7219990	total: 1.79s	remaining: 1.5s
545:	learn: 1.7215174	total: 1.8s	remaining: 1.49s
546:	learn: 1.7212851	total: 1.8s	remaining: 1.49s
547:	learn: 1.7210111	total: 1.8s	remaining: 1.49s
548:	learn: 1.7204384	total: 1.81s	remaining: 1.48s
549:	learn: 1.7195738	total: 1.81s	remaining: 1.48s
550:	learn: 1.7178995	total: 1.81s	remaining: 1.48s
551:	learn: 1.7167465	total: 1.82s	remaining: 1.47s
552:	learn: 1.7160833	total: 1.82s	remaining: 1.47s
553:	learn: 1.7154634	total: 1.82s	remaining: 1.47s
554:	learn: 1.7148687	total: 1.82s	remaining: 1.46s
555:	learn: 1.7144358	total: 1.83s	remaining: 1.46s
556:	learn: 1.7138888	total: 1.83s	remaining: 1.46s
557:	learn: 1.7135655	total: 1.83s	remaining: 1.45s
558:	learn: 1.7131455	total: 1.84s	remaining: 1.45s
559:	learn: 1.7121696	total: 1.84s	remaining: 1.45s
560:	learn: 1.7114

729:	learn: 1.5944440	total: 2.38s	remaining: 881ms
730:	learn: 1.5935774	total: 2.39s	remaining: 878ms
731:	learn: 1.5930970	total: 2.39s	remaining: 875ms
732:	learn: 1.5929530	total: 2.39s	remaining: 872ms
733:	learn: 1.5925554	total: 2.4s	remaining: 868ms
734:	learn: 1.5924232	total: 2.4s	remaining: 865ms
735:	learn: 1.5918312	total: 2.4s	remaining: 862ms
736:	learn: 1.5909258	total: 2.41s	remaining: 859ms
737:	learn: 1.5905505	total: 2.41s	remaining: 855ms
738:	learn: 1.5898107	total: 2.41s	remaining: 852ms
739:	learn: 1.5877515	total: 2.42s	remaining: 850ms
740:	learn: 1.5870121	total: 2.42s	remaining: 846ms
741:	learn: 1.5867676	total: 2.42s	remaining: 843ms
742:	learn: 1.5859175	total: 2.43s	remaining: 840ms
743:	learn: 1.5847454	total: 2.43s	remaining: 837ms
744:	learn: 1.5841735	total: 2.44s	remaining: 834ms
745:	learn: 1.5833041	total: 2.44s	remaining: 832ms
746:	learn: 1.5829718	total: 2.45s	remaining: 829ms
747:	learn: 1.5818820	total: 2.45s	remaining: 826ms
748:	learn: 1.5

...

965:	learn: 1.4306579	total: 3.23s	remaining: 114ms
966:	learn: 1.4299365	total: 3.24s	remaining: 111ms
967:	learn: 1.4296431	total: 3.24s	remaining: 107ms
968:	learn: 1.4290009	total: 3.24s	remaining: 104ms
969:	learn: 1.4282749	total: 3.25s	remaining: 100ms
970:	learn: 1.4280683	total: 3.25s	remaining: 97.1ms
971:	learn: 1.4272775	total: 3.25s	remaining: 93.8ms
972:	learn: 1.4269329	total: 3.26s	remaining: 90.4ms
973:	learn: 1.4255272	total: 3.26s	remaining: 87.1ms
974:	learn: 1.4249529	total: 3.26s	remaining: 83.7ms
975:	learn: 1.4243531	total: 3.27s	remaining: 80.4ms
976:	learn: 1.4239609	total: 3.27s	remaining: 77ms
977:	learn: 1.4237154	total: 3.27s	remaining: 73.7ms
978:	learn: 1.4224496	total: 3.28s	remaining: 70.3ms
979:	learn: 1.4221406	total: 3.28s	remaining: 67ms
980:	learn: 1.4215117	total: 3.28s	remaining: 63.6ms
981:	learn: 1.4209906	total: 3.29s	remaining: 60.3ms
982:	learn: 1.4205259	total: 3.29s	remaining: 56.9ms
983:	learn: 1.4196307	total: 3.29s	remaining: 53.6ms
98

175:	learn: 2.0738557	total: 390ms	remaining: 1.83s
176:	learn: 2.0730380	total: 392ms	remaining: 1.82s
177:	learn: 2.0712229	total: 394ms	remaining: 1.82s
178:	learn: 2.0699106	total: 395ms	remaining: 1.81s
179:	learn: 2.0684834	total: 397ms	remaining: 1.81s
180:	learn: 2.0651168	total: 399ms	remaining: 1.8s
181:	learn: 2.0635537	total: 400ms	remaining: 1.8s
182:	learn: 2.0626358	total: 402ms	remaining: 1.79s
183:	learn: 2.0594107	total: 404ms	remaining: 1.79s
184:	learn: 2.0570429	total: 406ms	remaining: 1.79s
185:	learn: 2.0558655	total: 407ms	remaining: 1.78s
186:	learn: 2.0537295	total: 409ms	remaining: 1.78s
187:	learn: 2.0528523	total: 411ms	remaining: 1.77s
188:	learn: 2.0521141	total: 412ms	remaining: 1.77s
189:	learn: 2.0498435	total: 414ms	remaining: 1.76s
190:	learn: 2.0491583	total: 416ms	remaining: 1.76s
191:	learn: 2.0480987	total: 417ms	remaining: 1.76s
192:	learn: 2.0471587	total: 419ms	remaining: 1.75s
193:	learn: 2.0449941	total: 421ms	remaining: 1.75s
194:	learn: 2.

407:	learn: 1.7987436	total: 783ms	remaining: 1.14s
408:	learn: 1.7976139	total: 784ms	remaining: 1.13s
409:	learn: 1.7966179	total: 786ms	remaining: 1.13s
410:	learn: 1.7955473	total: 788ms	remaining: 1.13s
411:	learn: 1.7944703	total: 790ms	remaining: 1.13s
412:	learn: 1.7936817	total: 791ms	remaining: 1.12s
413:	learn: 1.7927144	total: 793ms	remaining: 1.12s
414:	learn: 1.7909285	total: 795ms	remaining: 1.12s
415:	learn: 1.7900711	total: 796ms	remaining: 1.12s
416:	learn: 1.7883901	total: 798ms	remaining: 1.11s
417:	learn: 1.7871732	total: 800ms	remaining: 1.11s
418:	learn: 1.7860455	total: 801ms	remaining: 1.11s
419:	learn: 1.7855299	total: 803ms	remaining: 1.11s
420:	learn: 1.7852552	total: 805ms	remaining: 1.11s
421:	learn: 1.7849939	total: 806ms	remaining: 1.1s
422:	learn: 1.7843629	total: 808ms	remaining: 1.1s
423:	learn: 1.7823019	total: 810ms	remaining: 1.1s
424:	learn: 1.7810656	total: 811ms	remaining: 1.1s
425:	learn: 1.7792757	total: 813ms	remaining: 1.09s
426:	learn: 1.77

639:	learn: 1.6198604	total: 1.18s	remaining: 661ms
640:	learn: 1.6191595	total: 1.18s	remaining: 659ms
641:	learn: 1.6189354	total: 1.18s	remaining: 657ms
642:	learn: 1.6185870	total: 1.18s	remaining: 655ms
643:	learn: 1.6175703	total: 1.18s	remaining: 653ms
644:	learn: 1.6171103	total: 1.18s	remaining: 652ms
645:	learn: 1.6166131	total: 1.19s	remaining: 650ms
646:	learn: 1.6163127	total: 1.19s	remaining: 648ms
647:	learn: 1.6156949	total: 1.19s	remaining: 646ms
648:	learn: 1.6152267	total: 1.19s	remaining: 644ms
649:	learn: 1.6148007	total: 1.19s	remaining: 642ms
650:	learn: 1.6144663	total: 1.19s	remaining: 640ms
651:	learn: 1.6132952	total: 1.2s	remaining: 638ms
652:	learn: 1.6131002	total: 1.2s	remaining: 636ms
653:	learn: 1.6126019	total: 1.2s	remaining: 635ms
654:	learn: 1.6122475	total: 1.2s	remaining: 633ms
655:	learn: 1.6111049	total: 1.2s	remaining: 631ms
656:	learn: 1.6097181	total: 1.2s	remaining: 629ms
657:	learn: 1.6091902	total: 1.21s	remaining: 627ms
658:	learn: 1.6080

869:	learn: 1.4758178	total: 1.57s	remaining: 234ms
870:	learn: 1.4748583	total: 1.57s	remaining: 233ms
871:	learn: 1.4747591	total: 1.57s	remaining: 231ms
872:	learn: 1.4729538	total: 1.57s	remaining: 229ms
873:	learn: 1.4721833	total: 1.57s	remaining: 227ms
874:	learn: 1.4718885	total: 1.58s	remaining: 225ms
875:	learn: 1.4715382	total: 1.58s	remaining: 224ms
876:	learn: 1.4705216	total: 1.58s	remaining: 222ms
877:	learn: 1.4703272	total: 1.58s	remaining: 220ms
878:	learn: 1.4693250	total: 1.58s	remaining: 218ms
879:	learn: 1.4687791	total: 1.58s	remaining: 216ms
880:	learn: 1.4673506	total: 1.59s	remaining: 214ms
881:	learn: 1.4669840	total: 1.59s	remaining: 213ms
882:	learn: 1.4662909	total: 1.59s	remaining: 211ms
883:	learn: 1.4661402	total: 1.59s	remaining: 209ms
884:	learn: 1.4655658	total: 1.59s	remaining: 207ms
885:	learn: 1.4654652	total: 1.6s	remaining: 205ms
886:	learn: 1.4653675	total: 1.6s	remaining: 204ms
887:	learn: 1.4648611	total: 1.6s	remaining: 202ms
888:	learn: 1.4

In [18]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.081154099408758
1.0907348355217708
1.073224317231108
1.0940975721627586
1.0530170547379247
1.0946967173902278
0.5559656973482272
0.559068701795701
0.5239427039234275
0.5854382106656788
0.5503502603218104
0.5610286100345183


In [19]:
train_size = 7000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.055681
0:	learn: 3.1144044	total: 3.71ms	remaining: 3.71s
1:	learn: 3.0607946	total: 7.13ms	remaining: 3.56s
2:	learn: 3.0122215	total: 10.5ms	remaining: 3.48s
3:	learn: 2.9673771	total: 13.6ms	remaining: 3.39s
4:	learn: 2.9241772	total: 16.6ms	remaining: 3.31s
5:	learn: 2.8854828	total: 19.7ms	remaining: 3.27s
6:	learn: 2.8470189	total: 22.9ms	remaining: 3.25s
7:	learn: 2.8161751	total: 26.1ms	remaining: 3.24s
8:	learn: 2.7869043	total: 29.3ms	remaining: 3.23s
9:	learn: 2.7570194	total: 32.7ms	remaining: 3.23s
10:	learn: 2.7314993	total: 36.1ms	remaining: 3.24s
11:	learn: 2.7084458	total: 39.3ms	remaining: 3.23s
12:	learn: 2.6862980	total: 42.5ms	remaining: 3.23s
13:	learn: 2.6673648	total: 46.5ms	remaining: 3.28s
14:	learn: 2.6470519	total: 50ms	remaining: 3.29s
15:	learn: 2.6271639	total: 53.5ms	remaining: 3.29s
16:	learn: 2.6107205	total: 57.2ms	remaining: 3.31s
17:	learn: 2.5975259	total: 60.7ms	remaining: 3.31s
18:	learn: 2.5839499	total: 64.3ms	remaining: 

168:	learn: 2.0953047	total: 591ms	remaining: 2.9s
169:	learn: 2.0936638	total: 594ms	remaining: 2.9s
170:	learn: 2.0924656	total: 597ms	remaining: 2.9s
171:	learn: 2.0910331	total: 600ms	remaining: 2.89s
172:	learn: 2.0904330	total: 604ms	remaining: 2.89s
173:	learn: 2.0887901	total: 607ms	remaining: 2.88s
174:	learn: 2.0862733	total: 611ms	remaining: 2.88s
175:	learn: 2.0836077	total: 615ms	remaining: 2.88s
176:	learn: 2.0822543	total: 618ms	remaining: 2.87s
177:	learn: 2.0803670	total: 621ms	remaining: 2.87s
178:	learn: 2.0788154	total: 625ms	remaining: 2.87s
179:	learn: 2.0768894	total: 628ms	remaining: 2.86s
180:	learn: 2.0756126	total: 632ms	remaining: 2.86s
181:	learn: 2.0743396	total: 635ms	remaining: 2.85s
182:	learn: 2.0724283	total: 639ms	remaining: 2.85s
183:	learn: 2.0706518	total: 643ms	remaining: 2.85s
184:	learn: 2.0692560	total: 647ms	remaining: 2.85s
185:	learn: 2.0667122	total: 650ms	remaining: 2.85s
186:	learn: 2.0650050	total: 654ms	remaining: 2.84s
187:	learn: 2.0

336:	learn: 1.8691072	total: 1.19s	remaining: 2.34s
337:	learn: 1.8687594	total: 1.19s	remaining: 2.34s
338:	learn: 1.8667800	total: 1.2s	remaining: 2.33s
339:	learn: 1.8652897	total: 1.2s	remaining: 2.33s
340:	learn: 1.8643674	total: 1.2s	remaining: 2.33s
341:	learn: 1.8636569	total: 1.21s	remaining: 2.32s
342:	learn: 1.8628280	total: 1.21s	remaining: 2.32s
343:	learn: 1.8624704	total: 1.21s	remaining: 2.32s
344:	learn: 1.8606873	total: 1.22s	remaining: 2.31s
345:	learn: 1.8596102	total: 1.22s	remaining: 2.31s
346:	learn: 1.8583807	total: 1.22s	remaining: 2.3s
347:	learn: 1.8574177	total: 1.23s	remaining: 2.3s
348:	learn: 1.8558969	total: 1.23s	remaining: 2.3s
349:	learn: 1.8543117	total: 1.23s	remaining: 2.29s
350:	learn: 1.8539648	total: 1.24s	remaining: 2.29s
351:	learn: 1.8535600	total: 1.24s	remaining: 2.29s
352:	learn: 1.8515879	total: 1.25s	remaining: 2.28s
353:	learn: 1.8504967	total: 1.25s	remaining: 2.28s
354:	learn: 1.8501614	total: 1.25s	remaining: 2.27s
355:	learn: 1.8470

509:	learn: 1.7109667	total: 1.79s	remaining: 1.72s
510:	learn: 1.7103533	total: 1.79s	remaining: 1.71s
511:	learn: 1.7099604	total: 1.79s	remaining: 1.71s
512:	learn: 1.7093349	total: 1.8s	remaining: 1.71s
513:	learn: 1.7087089	total: 1.8s	remaining: 1.7s
514:	learn: 1.7075320	total: 1.8s	remaining: 1.7s
515:	learn: 1.7071072	total: 1.81s	remaining: 1.7s
516:	learn: 1.7059821	total: 1.81s	remaining: 1.69s
517:	learn: 1.7050543	total: 1.81s	remaining: 1.69s
518:	learn: 1.7040969	total: 1.82s	remaining: 1.69s
519:	learn: 1.7033169	total: 1.82s	remaining: 1.68s
520:	learn: 1.7028297	total: 1.82s	remaining: 1.68s
521:	learn: 1.7026257	total: 1.83s	remaining: 1.68s
522:	learn: 1.7017855	total: 1.83s	remaining: 1.67s
523:	learn: 1.7011946	total: 1.84s	remaining: 1.67s
524:	learn: 1.7007404	total: 1.84s	remaining: 1.66s
525:	learn: 1.7003317	total: 1.84s	remaining: 1.66s
526:	learn: 1.6990918	total: 1.85s	remaining: 1.66s
527:	learn: 1.6981347	total: 1.85s	remaining: 1.65s
528:	learn: 1.6967

681:	learn: 1.5930853	total: 2.39s	remaining: 1.11s
682:	learn: 1.5925695	total: 2.39s	remaining: 1.11s
683:	learn: 1.5922292	total: 2.39s	remaining: 1.11s
684:	learn: 1.5920168	total: 2.4s	remaining: 1.1s
685:	learn: 1.5913678	total: 2.4s	remaining: 1.1s
686:	learn: 1.5894351	total: 2.4s	remaining: 1.09s
687:	learn: 1.5891472	total: 2.41s	remaining: 1.09s
688:	learn: 1.5889025	total: 2.41s	remaining: 1.09s
689:	learn: 1.5887064	total: 2.42s	remaining: 1.08s
690:	learn: 1.5869240	total: 2.42s	remaining: 1.08s
691:	learn: 1.5863843	total: 2.42s	remaining: 1.08s
692:	learn: 1.5856135	total: 2.42s	remaining: 1.07s
693:	learn: 1.5852361	total: 2.43s	remaining: 1.07s
694:	learn: 1.5849392	total: 2.43s	remaining: 1.07s
695:	learn: 1.5839542	total: 2.44s	remaining: 1.06s
696:	learn: 1.5835562	total: 2.44s	remaining: 1.06s
697:	learn: 1.5827364	total: 2.44s	remaining: 1.06s
698:	learn: 1.5814578	total: 2.44s	remaining: 1.05s
699:	learn: 1.5812361	total: 2.45s	remaining: 1.05s
700:	learn: 1.581

...

220:	learn: 2.0184270	total: 788ms	remaining: 2.78s
221:	learn: 2.0167056	total: 792ms	remaining: 2.77s
222:	learn: 2.0150427	total: 795ms	remaining: 2.77s
223:	learn: 2.0129560	total: 798ms	remaining: 2.77s
224:	learn: 2.0109123	total: 802ms	remaining: 2.76s
225:	learn: 2.0099587	total: 805ms	remaining: 2.76s
226:	learn: 2.0083549	total: 808ms	remaining: 2.75s
227:	learn: 2.0068070	total: 812ms	remaining: 2.75s
228:	learn: 2.0061947	total: 815ms	remaining: 2.74s
229:	learn: 2.0041213	total: 818ms	remaining: 2.74s
230:	learn: 2.0037556	total: 821ms	remaining: 2.73s
231:	learn: 2.0030077	total: 824ms	remaining: 2.73s
232:	learn: 2.0020435	total: 827ms	remaining: 2.72s
233:	learn: 2.0016882	total: 830ms	remaining: 2.72s
234:	learn: 1.9991553	total: 834ms	remaining: 2.71s
235:	learn: 1.9985545	total: 837ms	remaining: 2.71s
236:	learn: 1.9969031	total: 840ms	remaining: 2.71s
237:	learn: 1.9942047	total: 844ms	remaining: 2.7s
238:	learn: 1.9934259	total: 847ms	remaining: 2.7s
239:	learn: 1.

392:	learn: 1.8392958	total: 1.38s	remaining: 2.14s
393:	learn: 1.8382931	total: 1.39s	remaining: 2.13s
394:	learn: 1.8369944	total: 1.39s	remaining: 2.13s
395:	learn: 1.8361378	total: 1.4s	remaining: 2.13s
396:	learn: 1.8355389	total: 1.4s	remaining: 2.13s
397:	learn: 1.8345727	total: 1.4s	remaining: 2.12s
398:	learn: 1.8343819	total: 1.41s	remaining: 2.12s
399:	learn: 1.8329280	total: 1.41s	remaining: 2.12s
400:	learn: 1.8320479	total: 1.41s	remaining: 2.11s
401:	learn: 1.8306367	total: 1.42s	remaining: 2.11s
402:	learn: 1.8296153	total: 1.42s	remaining: 2.11s
403:	learn: 1.8290038	total: 1.43s	remaining: 2.1s
404:	learn: 1.8280990	total: 1.43s	remaining: 2.1s
405:	learn: 1.8267152	total: 1.43s	remaining: 2.1s
406:	learn: 1.8248308	total: 1.44s	remaining: 2.09s
407:	learn: 1.8240662	total: 1.44s	remaining: 2.09s
408:	learn: 1.8230449	total: 1.44s	remaining: 2.08s
409:	learn: 1.8225109	total: 1.45s	remaining: 2.08s
410:	learn: 1.8220143	total: 1.45s	remaining: 2.08s
411:	learn: 1.8208

560:	learn: 1.6881887	total: 1.98s	remaining: 1.55s
561:	learn: 1.6870182	total: 1.98s	remaining: 1.55s
562:	learn: 1.6865230	total: 1.99s	remaining: 1.54s
563:	learn: 1.6863597	total: 1.99s	remaining: 1.54s
564:	learn: 1.6858471	total: 2s	remaining: 1.54s
565:	learn: 1.6854032	total: 2s	remaining: 1.53s
566:	learn: 1.6842500	total: 2s	remaining: 1.53s
567:	learn: 1.6839057	total: 2.01s	remaining: 1.53s
568:	learn: 1.6832050	total: 2.01s	remaining: 1.52s
569:	learn: 1.6818783	total: 2.01s	remaining: 1.52s
570:	learn: 1.6810453	total: 2.02s	remaining: 1.51s
571:	learn: 1.6799873	total: 2.02s	remaining: 1.51s
572:	learn: 1.6791975	total: 2.02s	remaining: 1.51s
573:	learn: 1.6790575	total: 2.03s	remaining: 1.5s
574:	learn: 1.6787433	total: 2.03s	remaining: 1.5s
575:	learn: 1.6783802	total: 2.04s	remaining: 1.5s
576:	learn: 1.6773495	total: 2.04s	remaining: 1.49s
577:	learn: 1.6766472	total: 2.04s	remaining: 1.49s
578:	learn: 1.6753927	total: 2.04s	remaining: 1.49s
579:	learn: 1.6745025	to

725:	learn: 1.5777554	total: 2.58s	remaining: 973ms
726:	learn: 1.5773386	total: 2.58s	remaining: 969ms
727:	learn: 1.5770956	total: 2.58s	remaining: 965ms
728:	learn: 1.5769879	total: 2.59s	remaining: 962ms
729:	learn: 1.5764586	total: 2.59s	remaining: 958ms
730:	learn: 1.5761284	total: 2.59s	remaining: 954ms
731:	learn: 1.5757327	total: 2.6s	remaining: 951ms
732:	learn: 1.5754033	total: 2.6s	remaining: 947ms
733:	learn: 1.5749758	total: 2.6s	remaining: 943ms
734:	learn: 1.5743105	total: 2.61s	remaining: 940ms
735:	learn: 1.5732879	total: 2.61s	remaining: 936ms
736:	learn: 1.5728435	total: 2.61s	remaining: 933ms
737:	learn: 1.5712652	total: 2.62s	remaining: 929ms
738:	learn: 1.5704921	total: 2.62s	remaining: 926ms
739:	learn: 1.5700533	total: 2.62s	remaining: 922ms
740:	learn: 1.5686792	total: 2.63s	remaining: 919ms
741:	learn: 1.5682168	total: 2.63s	remaining: 915ms
742:	learn: 1.5677628	total: 2.63s	remaining: 911ms
743:	learn: 1.5669231	total: 2.64s	remaining: 908ms
744:	learn: 1.5

897:	learn: 1.4826241	total: 3.18s	remaining: 361ms
898:	learn: 1.4822954	total: 3.18s	remaining: 357ms
899:	learn: 1.4820017	total: 3.18s	remaining: 354ms
900:	learn: 1.4815298	total: 3.19s	remaining: 350ms
901:	learn: 1.4814379	total: 3.19s	remaining: 347ms
902:	learn: 1.4812120	total: 3.19s	remaining: 343ms
903:	learn: 1.4810205	total: 3.2s	remaining: 340ms
904:	learn: 1.4804566	total: 3.2s	remaining: 336ms
905:	learn: 1.4802614	total: 3.21s	remaining: 333ms
906:	learn: 1.4799444	total: 3.21s	remaining: 329ms
907:	learn: 1.4792753	total: 3.21s	remaining: 326ms
908:	learn: 1.4785435	total: 3.22s	remaining: 322ms
909:	learn: 1.4783375	total: 3.22s	remaining: 319ms
910:	learn: 1.4781329	total: 3.22s	remaining: 315ms
911:	learn: 1.4779333	total: 3.23s	remaining: 311ms
912:	learn: 1.4776753	total: 3.23s	remaining: 308ms
913:	learn: 1.4771876	total: 3.23s	remaining: 304ms
914:	learn: 1.4766770	total: 3.24s	remaining: 301ms
915:	learn: 1.4764633	total: 3.24s	remaining: 297ms
916:	learn: 1.

In [20]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.0748187794209396
1.0768408400897715
1.0661009229819527
1.0867548199791677
1.05293879743142
1.0914585166223862
0.5648376327224399
0.577323353393397
0.5639701677307667
0.5961292284360047
0.5340963954640672
0.5526690185879637


In [21]:
train_size = 8000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.056868
0:	learn: 3.1024214	total: 3.29ms	remaining: 3.29s
1:	learn: 3.0458886	total: 6.24ms	remaining: 3.11s
2:	learn: 2.9940891	total: 9.14ms	remaining: 3.04s
3:	learn: 2.9475215	total: 12.3ms	remaining: 3.06s
4:	learn: 2.9038344	total: 15.4ms	remaining: 3.06s
5:	learn: 2.8665446	total: 18.7ms	remaining: 3.09s
6:	learn: 2.8320950	total: 21.8ms	remaining: 3.09s
7:	learn: 2.7969881	total: 24.5ms	remaining: 3.04s
8:	learn: 2.7668972	total: 27.8ms	remaining: 3.06s
9:	learn: 2.7410001	total: 30.9ms	remaining: 3.06s
10:	learn: 2.7133710	total: 34.2ms	remaining: 3.08s
11:	learn: 2.6846148	total: 37.4ms	remaining: 3.08s
12:	learn: 2.6627148	total: 40.8ms	remaining: 3.1s
13:	learn: 2.6437013	total: 44ms	remaining: 3.1s
14:	learn: 2.6243164	total: 47.5ms	remaining: 3.12s
15:	learn: 2.6079811	total: 50.9ms	remaining: 3.13s
16:	learn: 2.5910875	total: 55ms	remaining: 3.18s
17:	learn: 2.5749778	total: 58.4ms	remaining: 3.19s
18:	learn: 2.5604388	total: 61.9ms	remaining: 3.2s

163:	learn: 2.0978878	total: 595ms	remaining: 3.03s
164:	learn: 2.0969676	total: 599ms	remaining: 3.03s
165:	learn: 2.0941533	total: 602ms	remaining: 3.03s
166:	learn: 2.0920764	total: 606ms	remaining: 3.02s
167:	learn: 2.0913167	total: 610ms	remaining: 3.02s
168:	learn: 2.0888648	total: 613ms	remaining: 3.02s
169:	learn: 2.0872114	total: 617ms	remaining: 3.01s
170:	learn: 2.0848238	total: 620ms	remaining: 3.01s
171:	learn: 2.0838183	total: 624ms	remaining: 3s
172:	learn: 2.0826600	total: 627ms	remaining: 3s
173:	learn: 2.0811561	total: 630ms	remaining: 2.99s
174:	learn: 2.0793477	total: 634ms	remaining: 2.99s
175:	learn: 2.0785694	total: 637ms	remaining: 2.98s
176:	learn: 2.0775353	total: 640ms	remaining: 2.98s
177:	learn: 2.0764211	total: 644ms	remaining: 2.97s
178:	learn: 2.0735956	total: 647ms	remaining: 2.97s
179:	learn: 2.0725952	total: 651ms	remaining: 2.96s
180:	learn: 2.0716329	total: 654ms	remaining: 2.96s
181:	learn: 2.0689201	total: 658ms	remaining: 2.96s
182:	learn: 2.0671

328:	learn: 1.8726025	total: 1.19s	remaining: 2.43s
329:	learn: 1.8712206	total: 1.2s	remaining: 2.43s
330:	learn: 1.8707891	total: 1.2s	remaining: 2.42s
331:	learn: 1.8702405	total: 1.2s	remaining: 2.42s
332:	learn: 1.8687617	total: 1.21s	remaining: 2.42s
333:	learn: 1.8682873	total: 1.21s	remaining: 2.41s
334:	learn: 1.8676744	total: 1.21s	remaining: 2.41s
335:	learn: 1.8674253	total: 1.22s	remaining: 2.41s
336:	learn: 1.8670168	total: 1.22s	remaining: 2.4s
337:	learn: 1.8664833	total: 1.22s	remaining: 2.4s
338:	learn: 1.8662123	total: 1.23s	remaining: 2.39s
339:	learn: 1.8654735	total: 1.23s	remaining: 2.39s
340:	learn: 1.8638073	total: 1.24s	remaining: 2.39s
341:	learn: 1.8616180	total: 1.24s	remaining: 2.38s
342:	learn: 1.8606358	total: 1.24s	remaining: 2.38s
343:	learn: 1.8590812	total: 1.25s	remaining: 2.38s
344:	learn: 1.8568311	total: 1.25s	remaining: 2.37s
345:	learn: 1.8564429	total: 1.25s	remaining: 2.37s
346:	learn: 1.8556306	total: 1.26s	remaining: 2.36s
347:	learn: 1.853

494:	learn: 1.7142538	total: 1.79s	remaining: 1.82s
495:	learn: 1.7132578	total: 1.79s	remaining: 1.82s
496:	learn: 1.7125361	total: 1.8s	remaining: 1.82s
497:	learn: 1.7118399	total: 1.8s	remaining: 1.81s
498:	learn: 1.7110980	total: 1.8s	remaining: 1.81s
499:	learn: 1.7106735	total: 1.81s	remaining: 1.81s
500:	learn: 1.7103025	total: 1.81s	remaining: 1.8s
501:	learn: 1.7094284	total: 1.81s	remaining: 1.8s
502:	learn: 1.7092382	total: 1.82s	remaining: 1.8s
503:	learn: 1.7084142	total: 1.82s	remaining: 1.79s
504:	learn: 1.7081110	total: 1.82s	remaining: 1.79s
505:	learn: 1.7071058	total: 1.83s	remaining: 1.78s
506:	learn: 1.7069225	total: 1.83s	remaining: 1.78s
507:	learn: 1.7064810	total: 1.83s	remaining: 1.78s
508:	learn: 1.7057587	total: 1.84s	remaining: 1.77s
509:	learn: 1.7051774	total: 1.84s	remaining: 1.77s
510:	learn: 1.7046537	total: 1.85s	remaining: 1.77s
511:	learn: 1.7038038	total: 1.85s	remaining: 1.76s
512:	learn: 1.7036263	total: 1.85s	remaining: 1.76s
513:	learn: 1.7024

655:	learn: 1.6073047	total: 2.39s	remaining: 1.25s
656:	learn: 1.6070117	total: 2.39s	remaining: 1.25s
657:	learn: 1.6065169	total: 2.39s	remaining: 1.24s
658:	learn: 1.6060780	total: 2.4s	remaining: 1.24s
659:	learn: 1.6056656	total: 2.4s	remaining: 1.24s
660:	learn: 1.6051568	total: 2.4s	remaining: 1.23s
661:	learn: 1.6047966	total: 2.41s	remaining: 1.23s
662:	learn: 1.6037012	total: 2.41s	remaining: 1.23s
663:	learn: 1.6033299	total: 2.42s	remaining: 1.22s
664:	learn: 1.6022533	total: 2.42s	remaining: 1.22s
665:	learn: 1.6009947	total: 2.42s	remaining: 1.22s
666:	learn: 1.6003278	total: 2.43s	remaining: 1.21s
667:	learn: 1.5999700	total: 2.43s	remaining: 1.21s
668:	learn: 1.5989474	total: 2.43s	remaining: 1.2s
669:	learn: 1.5986660	total: 2.44s	remaining: 1.2s
670:	learn: 1.5982599	total: 2.44s	remaining: 1.2s
671:	learn: 1.5981250	total: 2.44s	remaining: 1.19s
672:	learn: 1.5977109	total: 2.45s	remaining: 1.19s
673:	learn: 1.5974338	total: 2.45s	remaining: 1.19s
674:	learn: 1.5970

...

191:	learn: 2.0577658	total: 693ms	remaining: 2.92s
192:	learn: 2.0571572	total: 697ms	remaining: 2.91s
193:	learn: 2.0560939	total: 700ms	remaining: 2.91s
194:	learn: 2.0552371	total: 704ms	remaining: 2.9s
195:	learn: 2.0545634	total: 707ms	remaining: 2.9s
196:	learn: 2.0526333	total: 711ms	remaining: 2.9s
197:	learn: 2.0499159	total: 715ms	remaining: 2.89s
198:	learn: 2.0488851	total: 718ms	remaining: 2.89s
199:	learn: 2.0469850	total: 722ms	remaining: 2.89s
200:	learn: 2.0461281	total: 725ms	remaining: 2.88s
201:	learn: 2.0452157	total: 729ms	remaining: 2.88s
202:	learn: 2.0434530	total: 732ms	remaining: 2.87s
203:	learn: 2.0416499	total: 736ms	remaining: 2.87s
204:	learn: 2.0407185	total: 739ms	remaining: 2.87s
205:	learn: 2.0390204	total: 743ms	remaining: 2.86s
206:	learn: 2.0372907	total: 747ms	remaining: 2.86s
207:	learn: 2.0356255	total: 751ms	remaining: 2.86s
208:	learn: 2.0331971	total: 754ms	remaining: 2.85s
209:	learn: 2.0320945	total: 758ms	remaining: 2.85s
210:	learn: 2.0

359:	learn: 1.8625235	total: 1.29s	remaining: 2.29s
360:	learn: 1.8615101	total: 1.29s	remaining: 2.29s
361:	learn: 1.8607551	total: 1.3s	remaining: 2.29s
362:	learn: 1.8601027	total: 1.3s	remaining: 2.28s
363:	learn: 1.8584122	total: 1.3s	remaining: 2.28s
364:	learn: 1.8570877	total: 1.31s	remaining: 2.28s
365:	learn: 1.8546955	total: 1.31s	remaining: 2.27s
366:	learn: 1.8541466	total: 1.31s	remaining: 2.27s
367:	learn: 1.8534983	total: 1.32s	remaining: 2.27s
368:	learn: 1.8524731	total: 1.32s	remaining: 2.26s
369:	learn: 1.8522645	total: 1.33s	remaining: 2.26s
370:	learn: 1.8511560	total: 1.33s	remaining: 2.25s
371:	learn: 1.8493006	total: 1.33s	remaining: 2.25s
372:	learn: 1.8481690	total: 1.34s	remaining: 2.25s
373:	learn: 1.8470971	total: 1.34s	remaining: 2.24s
374:	learn: 1.8457337	total: 1.34s	remaining: 2.24s
375:	learn: 1.8454874	total: 1.35s	remaining: 2.23s
376:	learn: 1.8446939	total: 1.35s	remaining: 2.23s
377:	learn: 1.8438893	total: 1.35s	remaining: 2.23s
378:	learn: 1.8

526:	learn: 1.7250338	total: 1.89s	remaining: 1.69s
527:	learn: 1.7231306	total: 1.89s	remaining: 1.69s
528:	learn: 1.7230039	total: 1.89s	remaining: 1.69s
529:	learn: 1.7222867	total: 1.9s	remaining: 1.68s
530:	learn: 1.7218748	total: 1.9s	remaining: 1.68s
531:	learn: 1.7211650	total: 1.91s	remaining: 1.68s
532:	learn: 1.7204003	total: 1.91s	remaining: 1.67s
533:	learn: 1.7188323	total: 1.91s	remaining: 1.67s
534:	learn: 1.7180268	total: 1.92s	remaining: 1.67s
535:	learn: 1.7170465	total: 1.93s	remaining: 1.67s
536:	learn: 1.7161486	total: 1.93s	remaining: 1.67s
537:	learn: 1.7145788	total: 1.94s	remaining: 1.67s
538:	learn: 1.7141709	total: 1.94s	remaining: 1.66s
539:	learn: 1.7133056	total: 1.95s	remaining: 1.66s
540:	learn: 1.7123273	total: 1.95s	remaining: 1.66s
541:	learn: 1.7099886	total: 1.96s	remaining: 1.65s
542:	learn: 1.7090605	total: 1.96s	remaining: 1.65s
543:	learn: 1.7081541	total: 1.96s	remaining: 1.65s
544:	learn: 1.7078158	total: 1.97s	remaining: 1.64s
545:	learn: 1.

734:	learn: 1.5834218	total: 2.69s	remaining: 969ms
735:	learn: 1.5830018	total: 2.69s	remaining: 966ms
736:	learn: 1.5826937	total: 2.69s	remaining: 962ms
737:	learn: 1.5824455	total: 2.7s	remaining: 958ms
738:	learn: 1.5817546	total: 2.7s	remaining: 955ms
739:	learn: 1.5807139	total: 2.71s	remaining: 951ms
740:	learn: 1.5801567	total: 2.71s	remaining: 947ms
741:	learn: 1.5792983	total: 2.71s	remaining: 944ms
742:	learn: 1.5787887	total: 2.72s	remaining: 940ms
743:	learn: 1.5786925	total: 2.72s	remaining: 936ms
744:	learn: 1.5782405	total: 2.72s	remaining: 932ms
745:	learn: 1.5774900	total: 2.73s	remaining: 929ms
746:	learn: 1.5771612	total: 2.73s	remaining: 925ms
747:	learn: 1.5759328	total: 2.73s	remaining: 921ms
748:	learn: 1.5756936	total: 2.74s	remaining: 918ms
749:	learn: 1.5751115	total: 2.74s	remaining: 914ms
750:	learn: 1.5748084	total: 2.75s	remaining: 910ms
751:	learn: 1.5745465	total: 2.75s	remaining: 907ms
752:	learn: 1.5738582	total: 2.75s	remaining: 903ms
753:	learn: 1.

900:	learn: 1.4941515	total: 3.28s	remaining: 361ms
901:	learn: 1.4936043	total: 3.29s	remaining: 357ms
902:	learn: 1.4933586	total: 3.29s	remaining: 354ms
903:	learn: 1.4931525	total: 3.29s	remaining: 350ms
904:	learn: 1.4924792	total: 3.3s	remaining: 346ms
905:	learn: 1.4922146	total: 3.3s	remaining: 343ms
906:	learn: 1.4913269	total: 3.31s	remaining: 339ms
907:	learn: 1.4910177	total: 3.31s	remaining: 335ms
908:	learn: 1.4907451	total: 3.31s	remaining: 332ms
909:	learn: 1.4905897	total: 3.32s	remaining: 328ms
910:	learn: 1.4903157	total: 3.32s	remaining: 324ms
911:	learn: 1.4902336	total: 3.32s	remaining: 321ms
912:	learn: 1.4894436	total: 3.33s	remaining: 317ms
913:	learn: 1.4886441	total: 3.33s	remaining: 313ms
914:	learn: 1.4876252	total: 3.33s	remaining: 310ms
915:	learn: 1.4872706	total: 3.34s	remaining: 306ms
916:	learn: 1.4870411	total: 3.34s	remaining: 302ms
917:	learn: 1.4867518	total: 3.34s	remaining: 299ms
918:	learn: 1.4862875	total: 3.35s	remaining: 295ms
919:	learn: 1.

In [22]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.0655710100020177
1.0845416803913148
1.0448074035014967
1.0869556889329275
1.0512829572854312
1.0602673198989183
0.5744861717327125
0.5905824234118516
0.5656354026057
0.5877890688954831
0.5710406375263413
0.5573833262241871


In [23]:
train_size = 9000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.057936
0:	learn: 3.2172700	total: 2.94ms	remaining: 2.94s
1:	learn: 3.1591950	total: 6.29ms	remaining: 3.14s
2:	learn: 3.1012588	total: 9.74ms	remaining: 3.24s
3:	learn: 3.0508995	total: 12.8ms	remaining: 3.2s
4:	learn: 3.0014706	total: 15.9ms	remaining: 3.16s
5:	learn: 2.9579251	total: 19ms	remaining: 3.15s
6:	learn: 2.9149212	total: 22.1ms	remaining: 3.13s
7:	learn: 2.8785011	total: 25.5ms	remaining: 3.16s
8:	learn: 2.8441011	total: 28.8ms	remaining: 3.17s
9:	learn: 2.8095852	total: 32ms	remaining: 3.17s
10:	learn: 2.7789435	total: 36ms	remaining: 3.24s
11:	learn: 2.7506234	total: 39.3ms	remaining: 3.24s
12:	learn: 2.7252410	total: 43ms	remaining: 3.26s
13:	learn: 2.7026903	total: 46.9ms	remaining: 3.3s
14:	learn: 2.6782485	total: 50.4ms	remaining: 3.31s
15:	learn: 2.6586483	total: 53.8ms	remaining: 3.31s
16:	learn: 2.6394976	total: 57.3ms	remaining: 3.31s
17:	learn: 2.6239743	total: 61.1ms	remaining: 3.33s
18:	learn: 2.6068552	total: 64.7ms	remaining: 3.34s
19

162:	learn: 2.1171632	total: 592ms	remaining: 3.04s
163:	learn: 2.1152621	total: 596ms	remaining: 3.04s
164:	learn: 2.1118934	total: 600ms	remaining: 3.03s
165:	learn: 2.1100682	total: 603ms	remaining: 3.03s
166:	learn: 2.1086835	total: 607ms	remaining: 3.03s
167:	learn: 2.1074724	total: 611ms	remaining: 3.03s
168:	learn: 2.1062754	total: 615ms	remaining: 3.02s
169:	learn: 2.1048004	total: 618ms	remaining: 3.02s
170:	learn: 2.1033361	total: 622ms	remaining: 3.02s
171:	learn: 2.1017679	total: 626ms	remaining: 3.01s
172:	learn: 2.1005700	total: 630ms	remaining: 3.01s
173:	learn: 2.0983863	total: 634ms	remaining: 3.01s
174:	learn: 2.0976807	total: 637ms	remaining: 3s
175:	learn: 2.0960550	total: 641ms	remaining: 3s
176:	learn: 2.0951063	total: 646ms	remaining: 3s
177:	learn: 2.0921150	total: 650ms	remaining: 3s
178:	learn: 2.0904335	total: 654ms	remaining: 3s
179:	learn: 2.0890917	total: 657ms	remaining: 2.99s
180:	learn: 2.0881610	total: 661ms	remaining: 2.99s
181:	learn: 2.0867735	total

326:	learn: 1.9146897	total: 1.19s	remaining: 2.46s
327:	learn: 1.9144796	total: 1.2s	remaining: 2.45s
328:	learn: 1.9135813	total: 1.2s	remaining: 2.45s
329:	learn: 1.9127593	total: 1.2s	remaining: 2.44s
330:	learn: 1.9110138	total: 1.21s	remaining: 2.44s
331:	learn: 1.9102454	total: 1.21s	remaining: 2.44s
332:	learn: 1.9095536	total: 1.21s	remaining: 2.43s
333:	learn: 1.9080792	total: 1.22s	remaining: 2.43s
334:	learn: 1.9073953	total: 1.22s	remaining: 2.43s
335:	learn: 1.9069629	total: 1.23s	remaining: 2.42s
336:	learn: 1.9067587	total: 1.23s	remaining: 2.42s
337:	learn: 1.9060952	total: 1.23s	remaining: 2.41s
338:	learn: 1.9052922	total: 1.24s	remaining: 2.41s
339:	learn: 1.9041523	total: 1.24s	remaining: 2.4s
340:	learn: 1.9030136	total: 1.24s	remaining: 2.4s
341:	learn: 1.9017911	total: 1.25s	remaining: 2.4s
342:	learn: 1.9011032	total: 1.25s	remaining: 2.4s
343:	learn: 1.9009040	total: 1.25s	remaining: 2.39s
344:	learn: 1.9001464	total: 1.26s	remaining: 2.39s
345:	learn: 1.89852

488:	learn: 1.7721397	total: 1.79s	remaining: 1.88s
489:	learn: 1.7712323	total: 1.8s	remaining: 1.87s
490:	learn: 1.7705846	total: 1.8s	remaining: 1.87s
491:	learn: 1.7692444	total: 1.81s	remaining: 1.87s
492:	learn: 1.7689292	total: 1.81s	remaining: 1.86s
493:	learn: 1.7683686	total: 1.81s	remaining: 1.86s
494:	learn: 1.7672692	total: 1.82s	remaining: 1.85s
495:	learn: 1.7670853	total: 1.82s	remaining: 1.85s
496:	learn: 1.7662720	total: 1.83s	remaining: 1.85s
497:	learn: 1.7655583	total: 1.83s	remaining: 1.84s
498:	learn: 1.7651050	total: 1.83s	remaining: 1.84s
499:	learn: 1.7647700	total: 1.84s	remaining: 1.84s
500:	learn: 1.7632542	total: 1.84s	remaining: 1.83s
501:	learn: 1.7627867	total: 1.85s	remaining: 1.83s
502:	learn: 1.7617297	total: 1.85s	remaining: 1.83s
503:	learn: 1.7615496	total: 1.85s	remaining: 1.82s
504:	learn: 1.7610074	total: 1.86s	remaining: 1.82s
505:	learn: 1.7602436	total: 1.86s	remaining: 1.82s
506:	learn: 1.7595484	total: 1.86s	remaining: 1.81s
507:	learn: 1.

656:	learn: 1.6516485	total: 2.45s	remaining: 1.28s
657:	learn: 1.6498116	total: 2.45s	remaining: 1.27s
658:	learn: 1.6496682	total: 2.46s	remaining: 1.27s
659:	learn: 1.6492609	total: 2.46s	remaining: 1.27s
660:	learn: 1.6488796	total: 2.46s	remaining: 1.26s
661:	learn: 1.6486438	total: 2.47s	remaining: 1.26s
662:	learn: 1.6479379	total: 2.47s	remaining: 1.25s
663:	learn: 1.6469194	total: 2.47s	remaining: 1.25s
664:	learn: 1.6466055	total: 2.48s	remaining: 1.25s
665:	learn: 1.6453567	total: 2.48s	remaining: 1.24s
666:	learn: 1.6447897	total: 2.48s	remaining: 1.24s
667:	learn: 1.6446681	total: 2.49s	remaining: 1.24s
668:	learn: 1.6442450	total: 2.49s	remaining: 1.23s
669:	learn: 1.6434837	total: 2.5s	remaining: 1.23s
670:	learn: 1.6429736	total: 2.5s	remaining: 1.23s
671:	learn: 1.6422637	total: 2.5s	remaining: 1.22s
672:	learn: 1.6415375	total: 2.51s	remaining: 1.22s
673:	learn: 1.6414110	total: 2.51s	remaining: 1.21s
674:	learn: 1.6411593	total: 2.51s	remaining: 1.21s
675:	learn: 1.6

...

328:	learn: 1.8917239	total: 1.19s	remaining: 2.43s
329:	learn: 1.8913243	total: 1.2s	remaining: 2.43s
330:	learn: 1.8901737	total: 1.2s	remaining: 2.43s
331:	learn: 1.8880309	total: 1.21s	remaining: 2.42s
332:	learn: 1.8870435	total: 1.21s	remaining: 2.42s
333:	learn: 1.8866452	total: 1.21s	remaining: 2.42s
334:	learn: 1.8858059	total: 1.22s	remaining: 2.41s
335:	learn: 1.8853573	total: 1.22s	remaining: 2.41s
336:	learn: 1.8843229	total: 1.22s	remaining: 2.41s
337:	learn: 1.8820536	total: 1.23s	remaining: 2.4s
338:	learn: 1.8810186	total: 1.23s	remaining: 2.4s
339:	learn: 1.8805145	total: 1.23s	remaining: 2.4s
340:	learn: 1.8796832	total: 1.24s	remaining: 2.39s
341:	learn: 1.8784520	total: 1.24s	remaining: 2.39s
342:	learn: 1.8774234	total: 1.25s	remaining: 2.39s
343:	learn: 1.8759545	total: 1.25s	remaining: 2.38s
344:	learn: 1.8742226	total: 1.25s	remaining: 2.38s
345:	learn: 1.8725979	total: 1.26s	remaining: 2.38s
346:	learn: 1.8714644	total: 1.26s	remaining: 2.37s
347:	learn: 1.870

491:	learn: 1.7510514	total: 1.79s	remaining: 1.85s
492:	learn: 1.7507559	total: 1.8s	remaining: 1.85s
493:	learn: 1.7500306	total: 1.8s	remaining: 1.85s
494:	learn: 1.7492463	total: 1.81s	remaining: 1.84s
495:	learn: 1.7490832	total: 1.81s	remaining: 1.84s
496:	learn: 1.7480410	total: 1.81s	remaining: 1.84s
497:	learn: 1.7461415	total: 1.82s	remaining: 1.83s
498:	learn: 1.7455848	total: 1.82s	remaining: 1.83s
499:	learn: 1.7450790	total: 1.82s	remaining: 1.82s
500:	learn: 1.7441313	total: 1.83s	remaining: 1.82s
501:	learn: 1.7433867	total: 1.83s	remaining: 1.82s
502:	learn: 1.7429972	total: 1.84s	remaining: 1.81s
503:	learn: 1.7421950	total: 1.84s	remaining: 1.81s
504:	learn: 1.7409289	total: 1.84s	remaining: 1.81s
505:	learn: 1.7406629	total: 1.85s	remaining: 1.8s
506:	learn: 1.7403416	total: 1.85s	remaining: 1.8s
507:	learn: 1.7392293	total: 1.85s	remaining: 1.79s
508:	learn: 1.7389686	total: 1.86s	remaining: 1.79s
509:	learn: 1.7376967	total: 1.86s	remaining: 1.79s
510:	learn: 1.73

655:	learn: 1.6431147	total: 2.39s	remaining: 1.25s
656:	learn: 1.6420992	total: 2.4s	remaining: 1.25s
657:	learn: 1.6417217	total: 2.4s	remaining: 1.25s
658:	learn: 1.6410708	total: 2.4s	remaining: 1.24s
659:	learn: 1.6397020	total: 2.41s	remaining: 1.24s
660:	learn: 1.6392970	total: 2.41s	remaining: 1.24s
661:	learn: 1.6386876	total: 2.42s	remaining: 1.23s
662:	learn: 1.6380264	total: 2.42s	remaining: 1.23s
663:	learn: 1.6375286	total: 2.42s	remaining: 1.23s
664:	learn: 1.6367151	total: 2.43s	remaining: 1.22s
665:	learn: 1.6364719	total: 2.43s	remaining: 1.22s
666:	learn: 1.6350613	total: 2.43s	remaining: 1.22s
667:	learn: 1.6342234	total: 2.44s	remaining: 1.21s
668:	learn: 1.6341125	total: 2.44s	remaining: 1.21s
669:	learn: 1.6329878	total: 2.44s	remaining: 1.2s
670:	learn: 1.6323440	total: 2.45s	remaining: 1.2s
671:	learn: 1.6320848	total: 2.45s	remaining: 1.2s
672:	learn: 1.6318476	total: 2.46s	remaining: 1.19s
673:	learn: 1.6316351	total: 2.46s	remaining: 1.19s
674:	learn: 1.6313

814:	learn: 1.5534252	total: 2.99s	remaining: 679ms
815:	learn: 1.5528827	total: 3s	remaining: 675ms
816:	learn: 1.5526611	total: 3s	remaining: 672ms
817:	learn: 1.5522942	total: 3s	remaining: 668ms
818:	learn: 1.5522046	total: 3.01s	remaining: 664ms
819:	learn: 1.5516903	total: 3.01s	remaining: 661ms
820:	learn: 1.5513119	total: 3.01s	remaining: 657ms
821:	learn: 1.5503926	total: 3.02s	remaining: 653ms
822:	learn: 1.5496081	total: 3.02s	remaining: 650ms
823:	learn: 1.5490917	total: 3.02s	remaining: 646ms
824:	learn: 1.5488458	total: 3.03s	remaining: 642ms
825:	learn: 1.5484930	total: 3.03s	remaining: 639ms
826:	learn: 1.5481039	total: 3.04s	remaining: 635ms
827:	learn: 1.5474277	total: 3.04s	remaining: 631ms
828:	learn: 1.5470150	total: 3.04s	remaining: 628ms
829:	learn: 1.5465392	total: 3.05s	remaining: 624ms
830:	learn: 1.5460409	total: 3.05s	remaining: 620ms
831:	learn: 1.5455914	total: 3.05s	remaining: 617ms
832:	learn: 1.5450699	total: 3.06s	remaining: 613ms
833:	learn: 1.5445493

975:	learn: 1.4797198	total: 3.59s	remaining: 88.3ms
976:	learn: 1.4792400	total: 3.59s	remaining: 84.6ms
977:	learn: 1.4789850	total: 3.6s	remaining: 80.9ms
978:	learn: 1.4785950	total: 3.6s	remaining: 77.3ms
979:	learn: 1.4783359	total: 3.6s	remaining: 73.6ms
980:	learn: 1.4778765	total: 3.61s	remaining: 69.9ms
981:	learn: 1.4777394	total: 3.61s	remaining: 66.2ms
982:	learn: 1.4774888	total: 3.62s	remaining: 62.5ms
983:	learn: 1.4772647	total: 3.62s	remaining: 58.9ms
984:	learn: 1.4771860	total: 3.62s	remaining: 55.2ms
985:	learn: 1.4765332	total: 3.63s	remaining: 51.5ms
986:	learn: 1.4763041	total: 3.63s	remaining: 47.8ms
987:	learn: 1.4758343	total: 3.63s	remaining: 44.1ms
988:	learn: 1.4751438	total: 3.64s	remaining: 40.5ms
989:	learn: 1.4749696	total: 3.64s	remaining: 36.8ms
990:	learn: 1.4747468	total: 3.64s	remaining: 33.1ms
991:	learn: 1.4745849	total: 3.65s	remaining: 29.4ms
992:	learn: 1.4732784	total: 3.65s	remaining: 25.7ms
993:	learn: 1.4719469	total: 3.65s	remaining: 22.

In [24]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.05561145873906
1.0571581620145192
1.0516129208014429
1.0540837861769046
1.0430853692648492
1.0721170554375838
0.6251275543843686
0.6636245855682567
0.6083989186870545
0.622687126033755
0.6196586754777538
0.611268466155023
